# Variant selection for CRISPEY-BAR library
This notebook details the variant selection process for several CRISPEY-BAR projects, including:
1. Epistasis
2. GxE (QTL pools)
3. GxE (ergosterol)
4. Others

In [1]:
import os, vcf, re
import multiprocessing as mp
from Bio import SeqIO
from collections import OrderedDict, namedtuple
import numpy as np
import pandas as pd

# Functions

In [2]:
def select_variants(genes_df, vcf_file, chrom_dict, upstream_length=500, downstream_length=500):
    '''
    given a dataframe of genes of interest, searches vcf for variants in and around that region
    can be parallelized by splitting genes and vcf by chromosome
    '''
    genes_df = genes_df.sort_values('start')
    # define start and end points to search depending on gene strand, and the upstream/downstream search parameters
    genes_df.loc[genes_df['strand']=='+', 'start_upstream'] = genes_df['start']-upstream_length
    genes_df.loc[genes_df['strand']=='+', 'end_downstream'] = genes_df['end']+downstream_length
    genes_df.loc[genes_df['strand']=='-', 'start_upstream'] = genes_df['start']-downstream_length
    genes_df.loc[genes_df['strand']=='-', 'end_downstream'] = genes_df['end']+upstream_length
    genes_df.loc[:,'start_upstream'] = genes_df['start_upstream'].astype(int)
    genes_df.loc[:,'end_downstream'] = genes_df['end_downstream'].astype(int)

    # read in vcf file to parse for variants
    allVars = vcf.Reader(filename=vcf_file)
    candVars = []
    for i, row in genes_df.iterrows():
        retrieve = allVars.fetch(chrom=chrom_dict[row['chrom']], 
                                 start=row['start_upstream']-1,
                                 end=row['end_downstream'])
        for rec in retrieve:
            rec.samples=[]
            rec.FORMAT=None
            candVars.append(rec)
    candVars = pd.Series(candVars).drop_duplicates().reset_index(drop=True)
    return candVars

def ngg_check(sequence, threshold):
    '''
    searches a Bio.seq object for NGG sequence around the center of the sequence
    set threshold on size of window to search for NGG (recommended: 10)
    returns a boolean of whether one or more NGG found
    '''
    hits=0
    midpt = (len(sequence)-1)//2
    for s in [sequence[midpt-2:midpt+threshold], sequence.reverse_complement()[midpt-2:midpt+threshold]]:
        matches = re.finditer('.GG', str(s), re.I)
        index_list = [m.start() for m in matches]
        hits+=len(index_list)
    if hits > 0:
        return True
    else:
        return False

def search_genome(sequence, genome):
    '''
    searches a genome (stored as dictionary of SeqRecords) for a Bio.Seq sequence (exact string match)
    returns number of occurences of the sequence in the genome
    '''
    hits = 0
    for scaffold in genome.values():
        # search both sequence and its reverse complement to find hits on forward and reverse strands
        for s in [sequence, sequence.reverse_complement()]:
            matches = re.finditer(str(s), str(scaffold.seq), re.I)
            index_list = [m.start() for m in matches]
            hits+=len(index_list)
    return hits

def filter_targetable_variants(candVars, genomes_list, threshold, window):
    '''
    Takes a list of VCF records, a list of genomes, a threshold for searching NGGs, and a window size for 
    flanking sequences, filters and sorts records into two lists, whether they are targetable in all genomes
    or just in the reference genome.
    '''
    ref_gen = genomes_list[0]
    candVars_allStrains=[]
    candVars_refOnly=[]
    
    for record in candVars:
        edits_in_ref_strain = False
        edits_in_nonref_strain = False
        ##### FILTERING #####
        # 1. Skip variant if near chromosome edge
        seq_window = ref_gen[record.CHROM][record.POS-window//2-1:record.POS+window//2]
        if len(seq_window) != window:
            continue

        # 2. is there an 'NGG' sequence around the center of the seq_window?
        if not ngg_check(sequence=seq_window.seq, threshold=threshold):
            continue

        # 3. check genomes if sequence flanking variant (seq_window) is unique
        if search_genome(seq_window.seq, ref_gen)==1: # reference genome
            edits_in_ref_strain = True
            # check remaining genomes to ensure seq_window is unique hit in all of them
            edits_in_nonref_strain = True
            for gen in genomes_list[1:]:
                if search_genome(seq_window.seq, gen)!=1:
                    edits_in_nonref_strain=False
                    break
        else:
            continue

        # 4. store variant info according to whether it can be edited in all strains, or just reference strain
        if edits_in_ref_strain:
            if edits_in_nonref_strain:
                candVars_allStrains.append(record)
            else:
                candVars_refOnly.append(record)
        else:
            print("Variant skipped: Cannot be edited in reference strain") # sanity check step: should not appear in output as filters in previous steps should remove all such instances
    
    return (candVars_allStrains, candVars_refOnly)

def find_targetable_variants(candVars, genomes_list, threshold, window):
    """
    UPGRADED FROM filter_targetable_variants
    Takes a list of VCF records, a list of genomes (at least 1), a threshold for searching NGGs, and a window size for search sequence.
    Filters records for NGG availability and targetability in each genome.
    Adds a list of integers (0/1) under "TGT" field in VCF INFO section. Number of digits corresponds to genomes provided
    For each genome, 1 indicates unique match for sequence is found, otherwise 0.
    NOTE: REMEMBER TO ADD "TGT" INFO TO VCF HEADER BEFORE WRITING RECORDS TO FILE
    header_info = collections.namedtuple('Info', ['id', 'num', 'type', 'desc', 'source', 'version'])
    header_info_tgt = header_info('TGT', '.', 'Integer', 'Targetability of variant in tested genomes', None, None)
    """
    ref_gen = genomes_list[0]

    record_list = []
    for record in candVars:
        tgt_code = [0]*len(genomes_list)
        ##### FILTERING #####
        # 1. Skip variant if near chromosome edge
        seq_window = ref_gen[record.CHROM][record.POS-window//2-1:record.POS+window//2]
        if len(seq_window) != window:
            continue

        # 2. is there an 'NGG' sequence around the center of the seq_window?
        if not ngg_check(sequence=seq_window.seq, threshold=threshold):
            continue

        # 3. check sequence flanking variant (seq_window) for unique match in genome
        for i in range(len(genomes_list)):
            gen = genomes_list[i]
            num_of_matches = search_genome(seq_window.seq, gen)
            if num_of_matches==1: # unique match
                tgt_code[i]=1
        # sequence window must match at least one genome in order to be written
        if sum(tgt_code)==0:
#             print("Variant at chrom {}, pos {} skipped: No unique matches found in tested genome(s)".format(record.CHROM, record.POS))
            continue
        # 4. add TGT code to variant record, add to record list
        else:
            record.INFO['TGT'] = tgt_code
            record_list.append(record)
        
    return record_list

##### FUNCTIONS TO ANNOTATE VARIANTS, BASED ON EILON'S OLIGO DESIGN PIPELINE #####
def get_genes_len(input_gff_filename):
    with open(input_gff_filename) as f:
        gff_lines = f.readlines()
    gff_lines = [l for l in gff_lines if len(l.split('\t'))>=5 ]    
    gff_lines = [l for l in gff_lines if l.split('\t')[2] == 'gene' ]
    genes_ids = [l.split('ID=')[1].split(';Name=')[0] for l in gff_lines]
    genes_lens = [ int(l.split('\t')[4]) - int(l.split('\t')[3]) + 1 for l in gff_lines]

    gene_le_df = pd.DataFrame({'Gene' : genes_ids, 'len_bp' : genes_lens})
    gene_le_df.set_index(keys='Gene', drop=False, inplace=True)
    gene_le_df.index.name=''
    gene_le_df.drop_duplicates(inplace=True)

    return(gene_le_df)

def get_cdss_len(input_gff_filename):
    with open(input_gff_filename) as f:
        gff_lines = f.readlines()
    gff_lines = [l for l in gff_lines if len(l.split('\t'))>=5 ]
    gff_lines = [l for l in gff_lines if l.split('\t')[2] == 'CDS' ]
    genes_ids = [l.split('Parent=')[1].split(';Name=')[0] for l in gff_lines]
    genes_lens = [ int(l.split('\t')[4]) - int(l.split('\t')[3]) + 1 for l in gff_lines]

    gene_le_df = pd.DataFrame({'Gene' : genes_ids, 'len_bp' : genes_lens})
    gene_le_df.set_index(keys='Gene', drop=False, inplace=True)
    gene_le_df.index.name=''
    gene_le_df = gene_le_df.groupby('Gene').sum()
    gene_le_df['Gene'] = gene_le_df.index
    gene_le_df.index.name=''
    gene_le_df.drop_duplicates(inplace=True)

    return(gene_le_df)

def get_dubious_genes(input_gff_filename):
    with open(input_gff_filename) as f:
        gff_lines = f.readlines()
    gff_lines = [l for l in gff_lines if len(l.split('\t'))>=5 ]
    gff_lines = [l for l in gff_lines if (l.split('\t')[2] == 'gene') & (len(re.findall('dubious', l, flags=re.I))>0)]
    genes_ids = [l.split('ID=')[1].split(';Name=')[0] for l in gff_lines]
    
    return genes_ids

def annotate_variants_by_VEPoutput(vep_input_vcf_filename, vep_output_filename, input_gff_filename, annotated_res_output_filename, id_colname = 'var_id'):
    cdss_len_df = get_cdss_len(input_gff_filename)
    genes_len_df = get_genes_len(input_gff_filename)
    dubious_genes = get_dubious_genes(input_gff_filename)
    
    vep_df = pd.read_table(vep_output_filename, sep='\t', na_values = "-", low_memory=False)
    vep_df = vep_df.loc[~vep_df.Gene.isin(dubious_genes),]
    vep_df.rename(columns={"#Uploaded_variation" : id_colname}, inplace=True)
    
    # prepare annotated variants df using VCF supplied to VEP
    variants_annotated_df = pd.Series([rec for rec in vcf.Reader(filename=vep_input_vcf_filename)])
    var_id_list = []
    assemble = []
    columns = ['CHROM', 'POS', 'REF', 'ALT', 'AC', 'AN' ,'AF']
    for rec in variants_annotated_df:
        var_id_list.append(rec.ID)
        assemble.append((rec.CHROM, rec.POS, rec.REF, rec.ALT[0], rec.INFO['AC'][0], rec.INFO['AN'], rec.INFO['AF'][0]))
    variants_annotated_df = pd.DataFrame.from_records(assemble, index=var_id_list, columns=columns)
    variants_annotated_df.index.name = id_colname
    
    # parse VEP output text file by ID for annotations
    entry_cnt = 0
    for cur_id, anno in vep_df.groupby(id_colname):
        entry_cnt +=1
        if (entry_cnt % 5000 == 0):
            print("parsing entry: %d"% (entry_cnt))
        
        # all up/down stream genes (VEP default search range is 5kb from variant position)
        cur_upstream_all_str =  '|'.join([str(int(d)) for d in anno.DISTANCE[anno.Consequence.isin(['upstream_gene_variant' ])].values if not np.isnan(d)])
        cur_downstream_all_str =  '|'.join([str(int(d)) for d in anno.DISTANCE[anno.Consequence.isin(['downstream_gene_variant' ])].values if not np.isnan(d)])
        
        variants_annotated_df.loc[cur_id, 'upstream_distance_str'] = cur_upstream_all_str
        variants_annotated_df.loc[cur_id, 'downstream_distance_str'] = cur_downstream_all_str

        is_up_down_stream = anno.Consequence.isin(['upstream_gene_variant' ,'downstream_gene_variant'])
        
        # identify intergenic variants
        if np.all(is_up_down_stream):
            left_genes = ( ((anno.Consequence =='upstream_gene_variant') & (anno.STRAND<0)) |
                          ((anno.Consequence =='downstream_gene_variant') & (anno.STRAND>0)) )

            right_genes = ( ((anno.Consequence =='upstream_gene_variant') & (anno.STRAND>0)) |
                              ((anno.Consequence =='downstream_gene_variant') & (anno.STRAND<0)) )

            if (left_genes.sum() > 0):
                ind_of_closest_left = anno.loc[left_genes,'DISTANCE'].idxmin()
                
                variants_annotated_df.loc[cur_id, 'closest_gene1_Gene_Name'] = anno['SYMBOL'][ind_of_closest_left]
                variants_annotated_df.loc[cur_id, 'closest_gene1_Gene_ID'] = anno['Gene'][ind_of_closest_left]
                variants_annotated_df.loc[cur_id, 'closest_gene1_Annotation'] = anno['Consequence'][ind_of_closest_left]
                variants_annotated_df.loc[cur_id, 'closest_gene1_Distance'] = anno['DISTANCE'][ind_of_closest_left]

            if (right_genes.sum() > 0):
                ind_of_closest_right = anno.loc[right_genes,'DISTANCE'].idxmin()
                
                variants_annotated_df.loc[cur_id, 'closest_gene2_Gene_Name'] = anno['SYMBOL'][ind_of_closest_right]
                variants_annotated_df.loc[cur_id, 'closest_gene2_Gene_ID'] = anno['Gene'][ind_of_closest_right]
                variants_annotated_df.loc[cur_id, 'closest_gene2_Annotation'] = anno['Consequence'][ind_of_closest_right]
                variants_annotated_df.loc[cur_id, 'closest_gene2_Distance'] = anno['DISTANCE'][ind_of_closest_right]

            # determine type of noncoding variant
            # Caution: variants without genes on one side will be labelled as "intergenic"
            cur_region = 'intergenic'
            if ((left_genes.sum() > 0) and (right_genes.sum() > 0)): 
                if (anno['Consequence'][ind_of_closest_left] == 'upstream_gene_variant'):
                    if (anno['Consequence'][ind_of_closest_right] == 'upstream_gene_variant'):
                        cur_region = 'bidirectional_promoter'
                    else:
                        cur_region = 'unidirectional_promoter'
                else:
                    if (anno['Consequence'][ind_of_closest_right] == 'upstream_gene_variant'):
                        cur_region = 'unidirectional_promoter'

            variants_annotated_df.loc[cur_id, 'region'] = cur_region

            # index of closest gene
            sel_row = anno.DISTANCE.idxmin()
        
        # for variants that land in a gene, select annotation with highest impact
        else:
            anno = anno.loc[~is_up_down_stream,:]
            if (anno.IMPACT == 'HIGH').sum()>0:
                anno = anno.loc[anno.IMPACT == 'HIGH',:]
            elif (anno.IMPACT == 'MODERATE').sum()>0:
                anno = anno.loc[anno.IMPACT == 'MODERATE',:]
            elif (anno.IMPACT == 'LOW').sum()>0:
                anno = anno.loc[anno.IMPACT == 'LOW',:]
            elif (anno.IMPACT == 'MODIFIER').sum()>0:
                anno = anno.loc[anno.IMPACT == 'MODIFIER',:] #NOTE: includes introns, noncoding transcripts within length of gene
            
            else:
                raise ValueError(anno)

            sel_row = anno.index[0]
            variants_annotated_df.loc[cur_id, 'region'] = anno['Consequence'][sel_row]

        # closest annotation recorded
        variants_annotated_df.loc[cur_id, 'Annotation'] = anno['Consequence'][sel_row]
        variants_annotated_df.loc[cur_id, 'Annotation_Impact'] = anno['IMPACT'][sel_row]
        variants_annotated_df.loc[cur_id, 'Gene_Name'] = anno['SYMBOL'][sel_row]
        variants_annotated_df.loc[cur_id, 'Gene_ID'] = anno['Gene'][sel_row]
        variants_annotated_df.loc[cur_id, 'Transcript_BioType'] = anno['BIOTYPE'][sel_row]
        variants_annotated_df.loc[cur_id, 'Existing_variation'] = anno['Existing_variation'][sel_row]
        variants_annotated_df.loc[cur_id, 'HGVSc'] = anno['HGVSc'][sel_row]
        variants_annotated_df.loc[cur_id, 'HGVSp'] = anno['HGVSp'][sel_row]
        variants_annotated_df.loc[cur_id, 'SWISSPROT'] = anno['SWISSPROT'][sel_row]
        variants_annotated_df.loc[cur_id, 'UNIPARC'] = anno['UNIPARC'][sel_row]
        variants_annotated_df.loc[cur_id, 'BLOSUM62'] = anno['BLOSUM62'][sel_row]
        variants_annotated_df.loc[cur_id, 'HGVSp'] = anno['HGVSp'][sel_row]
#         variants_annotated_df.set_value(index=cur_id,col='VEP_CSN',value=anno['CSN'][sel_row])
        
        # store cDNA and CDS positions
        cdna_pos = anno['cDNA_position'][sel_row]
        cds_pos = anno['CDS_position'][sel_row]
    
        # add additional info for missense and synonymous variants
        if (anno['Consequence'][sel_row] in ['synonymous_variant','missense_variant']):
            # because variants were extracted from 1011 genomes GVCF, some SNPs have trailing bases.
            # Calculate the correct cDNA and CDS positions if so 
            if '-' in cdna_pos:
                cdna_pos = cdna_pos.split('-')[0]
            if '-' in cds_pos:
                cds_pos = cds_pos.split('-')[0]
                
            
            if anno['Gene'][sel_row] in genes_len_df.index:
                cur_gene_len = int(genes_len_df.loc[anno['Gene'][sel_row],'len_bp'])
                variants_annotated_df.loc[cur_id, 'cDNA_len'] = cur_gene_len
                variants_annotated_df.loc[cur_id, 'cDNA_frac'] = int(cdna_pos)/cur_gene_len
            
            if anno['Gene'][sel_row] in cdss_len_df.index:
                cur_cds_len = int(cdss_len_df.loc[anno['Gene'][sel_row],'len_bp'])
                variants_annotated_df.loc[cur_id, 'CDS_len'] = cur_cds_len
                variants_annotated_df.loc[cur_id, 'CDS_frac'] = int(cds_pos)/cur_cds_len
                variants_annotated_df.loc[cur_id, 'AA_len'] = cur_cds_len/3
                variants_annotated_df.loc[cur_id, 'AA_frac'] = int(cds_pos)/cur_cds_len
                
            variants_annotated_df.loc[cur_id, 'cDNA_pos'] = cdna_pos
            variants_annotated_df.loc[cur_id, 'CDS_pos'] = cds_pos
            variants_annotated_df.loc[cur_id, 'AA_pos'] = anno['Protein_position'][sel_row]
            variants_annotated_df.loc[cur_id, 'Amino_acids'] = anno['Amino_acids'][sel_row]
            variants_annotated_df.loc[cur_id, 'Codons'] = anno['Codons'][sel_row]
            variants_annotated_df.loc[cur_id, 'DOMAINS'] = anno['DOMAINS'][sel_row]

    variants_annotated_df.to_csv(annotated_res_output_filename, sep='\t', header=True, index=True)
    
    return(variants_annotated_df)

###############################################################################################################
def find_pool_size_options(num_of_oligos, min_pool_size, max_pool_size, complete_pool_size=121):
    """
    helper function to find number of pools required to fit a certain number of oligos,
    as well as the average size of the pool
    """
    print('Total number of oligos to fit:', num_of_oligos)

    # num of pools to use
    num_of_pools_lower = num_of_oligos//max_pool_size
    num_of_pools_upper = num_of_oligos//min_pool_size
    display((num_of_pools_lower, num_of_pools_upper))

    # explore pooling options for all possible pool configurations
    for num_of_pools in range(num_of_pools_lower, num_of_pools_upper+1):
        if num_of_pools == 0:
            continue
        print("Use {} pools:".format(num_of_pools))
        oligos_per_pool = min((num_of_oligos // num_of_pools), max_pool_size)
        print("Number of oligos per pool:", oligos_per_pool)
        print("Number of oligos leftover:", num_of_oligos - num_of_pools*oligos_per_pool)
        print("Number of technical oligos:", (complete_pool_size - oligos_per_pool) * num_of_pools)
        print()
    
    # return list of possible pool sizes
    return np.arange(num_of_pools_lower, num_of_pools_upper+1)

## Count num. of genetic interactions per gene in Costanzo et al. (2016) data

In [3]:
# set working directory
working_dir = "/home/users/rang/scratch/yeast/genetic_interactions/costanzo_2016/"
# list of files to parse for genetic interactions
file_list = ['SGA_ExE.txt', 'SGA_ExN_NxE.txt', 'SGA_NxN.txt']
# summary of genetic interactions output file
out_file = working_dir+'genetic_interactions_summary_by_gene.txt'
os.chdir(working_dir)

In [4]:
df = []
for file in file_list:
    df.append(pd.read_csv(working_dir+file, sep='\t', header = 0))
df = pd.concat(df, axis=0, join='inner')
df.columns = ['query_strain', 'query_allele', 'array_strain', 'array_allele', 'arraytype', 'genetic_interaction_score', 'pval', 'query_smf', 'array_smf', 'dmf', 'dmf_std']
df['genetic_interaction_score_abs'] = df['genetic_interaction_score'].abs()
df['query_gene_id'] = [x.split('_')[0] for x in df['query_strain']]
df['query_gene_name'] = [x.split('-')[0] for x in df['query_allele']]
df['array_gene_id'] = [x.split('_')[0] for x in df['array_strain']]
df['array_gene_name'] = [x.split('-')[0] for x in df['array_allele']]
df['gene_pair'] = df.apply(lambda x: '_'.join(sorted([x['query_gene_id'], x['array_gene_id']])), axis=1)

# get list of genes
all_genes = set(df.query_gene_id).union(set(df.array_gene_id))

# filter down to significant interactions only
pval_threshold=0.05
interaction_threshold=0.08
df = df.query('genetic_interaction_score_abs>@interaction_threshold & pval<@pval_threshold')
display(df.shape)
display(df.head())

(890107, 17)

,query_strain,query_allele,array_strain,array_allele,arraytype,genetic_interaction_score,pval,query_smf,array_smf,dmf,dmf_std,genetic_interaction_score_abs,query_gene_id,query_gene_name,array_gene_id,array_gene_name,gene_pair
1,YAL001C_tsq508,tfc3-g349e,YBL026W_tsa1065,lsm2-5001,TSA30,-0.3529,3.591000e-06,0.8285,0.9408,0.4266,0.0790,0.3529,YAL001C,tfc3,YBL026W,lsm2,YAL001C_YBL026W
6,YAL001C_tsq508,tfc3-g349e,YBL034C_tsa950,stu1-7,TSA30,-0.1294,1.931000e-02,0.8285,0.6690,0.4249,0.0482,0.1294,YAL001C,tfc3,YBL034C,stu1,YAL001C_YBL034C
15,YAL001C_tsq508,tfc3-g349e,YBL097W_tsa510,brn1-9,TSA30,-0.0808,5.582000e-15,0.8285,0.5464,0.3719,0.0077,0.0808,YAL001C,tfc3,YBL097W,brn1,YAL001C_YBL097W
24,YAL001C_tsq508,tfc3-g349e,YBR029C_tsa1063,cds1-5001,TSA30,-0.1173,8.243000e-05,0.8285,0.9007,0.6289,0.0226,0.1173,YAL001C,tfc3,YBR029C,cds1,YAL001C_YBR029C
30,YAL001C_tsq508,tfc3-g349e,YBR060C_tsa311,orc2-2,TSA30,0.2516,4.729000e-12,0.8285,0.7384,0.8634,0.0362,0.2516,YAL001C,tfc3,YBR060C,orc2,YAL001C_YBR060C


In [5]:
# find strongest interaction in each pair of genes
gene_pairs_df = df.groupby('gene_pair').apply(lambda x: x.sort_values('genetic_interaction_score_abs', ascending=False).iloc[0])
display(gene_pairs_df.shape)
display(gene_pairs_df.head())

(780653, 17)

,query_strain,query_allele,array_strain,array_allele,arraytype,genetic_interaction_score,pval,query_smf,array_smf,dmf,dmf_std,genetic_interaction_score_abs,query_gene_id,query_gene_name,array_gene_id,array_gene_name,gene_pair
gene_pair,,,,,,,,,,,,,,,,,
YAL001C_YBL010C,YAL001C_tsq508,tfc3-g349e,YBL010C_dma88,ybl010c,DMA30,-0.0818,3.430000e-02,0.8285,0.9870,0.7360,0.0361,0.0818,YAL001C,tfc3,YBL010C,ybl010c,YAL001C_YBL010C
YAL001C_YBL026W,YAL001C_tsq508,tfc3-g349e,YBL026W_tsa1065,lsm2-5001,TSA30,-0.3529,3.591000e-06,0.8285,0.9408,0.4266,0.0790,0.3529,YAL001C,tfc3,YBL026W,lsm2,YAL001C_YBL026W
YAL001C_YBL034C,YAL001C_tsq508,tfc3-g349e,YBL034C_tsa950,stu1-7,TSA30,-0.1294,1.931000e-02,0.8285,0.6690,0.4249,0.0482,0.1294,YAL001C,tfc3,YBL034C,stu1,YAL001C_YBL034C
YAL001C_YBL097W,YAL001C_tsq508,tfc3-g349e,YBL097W_tsa510,brn1-9,TSA30,-0.0808,5.582000e-15,0.8285,0.5464,0.3719,0.0077,0.0808,YAL001C,tfc3,YBL097W,brn1,YAL001C_YBL097W
YAL001C_YBR029C,YAL001C_tsq508,tfc3-g349e,YBR029C_tsa1063,cds1-5001,TSA30,-0.1173,8.243000e-05,0.8285,0.9007,0.6289,0.0226,0.1173,YAL001C,tfc3,YBR029C,cds1,YAL001C_YBR029C


In [6]:
# for each gene, record number of positive and negative interactions that meet interaction and pvalue thresholds
gene_list = []
positive_interactions = []
negative_interactions = []
for gene in all_genes:
    sub = gene_pairs_df.query('query_gene_id==@gene | array_gene_id==@gene')
    gene_list.append(gene)
    positive_interactions.append(len(sub[sub['genetic_interaction_score']>interaction_threshold]))
    negative_interactions.append(len(sub[sub['genetic_interaction_score']<interaction_threshold*-1]))

num_of_interactions_df = pd.DataFrame({'positive_interactions':positive_interactions, 'negative_interactions':negative_interactions}, index=gene_list)
num_of_interactions_df = num_of_interactions_df.loc[num_of_interactions_df.sum(axis=1).sort_values(ascending=False).index]
num_of_interactions_df.index.name = 'gene_id'
display(num_of_interactions_df.shape)
display(num_of_interactions_df.head())

num_of_interactions_df.to_csv(out_file, sep='\t', header=True, index=True)

(5707, 2)

,positive_interactions,negative_interactions
gene_id,,
YFL039C,1324,2220
YFL034C-B,1150,1897
YJR076C,749,1342
YBL105C,841,1139
YER157W,925,967


## Read S288C GFF file and extract gene information

In [7]:
gene_info_df = {}
gff_file = '/home/users/rang/yeast/genomes/saccharomyces_cerevisiae_R64-1-1_20110208.gff'
char_map = {'%20' : ' ',
            '%2C' : ',',
            '%2F' : '/',
            '%3B' : ';'}

with open(gff_file, 'r') as gff:
    line = gff.readline()
    while line.rstrip() != '###':
        while line[0] == "#":
            line = gff.readline()
            
        chrom, _, feature, start, end, _, strand, _, info = line.rstrip().split('\t')
        if feature=='gene':
            info = {x.split('=')[0] : x.split('=')[1] for x in info.split(';')}
            gene_id = info['ID']
            if 'gene' in info:
                gene_name = info['gene']
            else:
                gene_name = np.nan
                
            if 'Note' in info:
                desc = info['Note']
                for c, r in char_map.items():
                    desc = desc.replace(c, r)
            else:
                desc = np.nan
            # store gene info
            gene_info_df[gene_id] = [chrom, int(start), int(end), strand, gene_name, desc]
        line = gff.readline()
        
gene_info_df = pd.DataFrame.from_dict(gene_info_df, orient='index', columns=['chrom', 'start', 'end', 'strand', 'gene_name', 'description'])

In [8]:
# merge GFF info with genetic interactions df
num_of_interactions_file = "/home/users/rang/scratch/yeast/genetic_interactions/costanzo_2016/genetic_interactions_summary_by_gene.txt"
num_of_interactions_df = pd.read_csv(num_of_interactions_file, sep='\t', header = 0, index_col=0)
num_of_interactions_df = num_of_interactions_df.join(gene_info_df).sort_values(['chrom', 'start'])
num_of_interactions_df = num_of_interactions_df.loc[num_of_interactions_df[['positive_interactions', 'negative_interactions']].sum(axis=1).sort_values(ascending=False).index]
display(num_of_interactions_df.head())

num_of_interactions_df.to_csv(out_file.replace('.txt', '_annotated.txt'), sep='\t', header=True, index=True)

,positive_interactions,negative_interactions,chrom,start,end,strand,gene_name,description
gene_id,,,,,,,,
YFL039C,1324,2220,chrVI,53260.0,54696.0,-,ACT1,"Actin, structural protein involved in cell pol..."
YFL034C-B,1150,1897,chrVI,63016.0,63993.0,-,MOB2,"Component of the RAM signaling network, that a..."
YJR076C,749,1342,chrX,575354.0,576601.0,-,CDC11,Component of the septin ring of the mother-bud...
YBL105C,841,1139,chrII,14241.0,17696.0,-,PKC1,Protein serine/threonine kinase essential for ...
YER157W,925,967,chrV,484788.0,487193.0,+,COG3,Essential component of the conserved oligomeri...


# Selecting natural variants for yeast epistasis project 
Screen natural variation in "hub" genes enriched for gene-gene interactions (Costanzo et al., 2016) to identify genetic background effects on growth fitness.

In [9]:
# set working directory
working_dir = "/home/users/rang/scratch/yeast/genetic_interactions/costanzo_2016/"
# summary of genetic interactions output file
genetic_interactions_summary_file = working_dir+'genetic_interactions_summary_by_gene.txt'
os.chdir(working_dir)

## Select top genes with most genetic interactions

In [10]:
genes_cutoff = 800

num_of_interactions_df = pd.read_csv(genetic_interactions_summary_file.replace('.txt', '_annotated.txt'), sep='\t', header = 0, index_col=0)
num_of_interactions_df = num_of_interactions_df.dropna().astype({'start':'int', 'end':'int'})

# remove dubious orfs
dubious_orfs_file = working_dir+'dubious_orfs.tsv'
dubious_orfs = pd.read_csv(dubious_orfs_file, sep='\t', header=None, names=['gene_id', 'description'])
num_of_interactions_df = num_of_interactions_df.query('~gene_id.isin(@dubious_orfs.gene_id)')

# genes in excluded list
genes_to_exclude = ['YDL227C', # HO
                    'YOR202W', # HIS3
                    'YEL021W', # URA3
                    'YCL018W', # LEU2
                    'YBR115C', # LYS2
                    'YBR020W', # GAL1
                    'YDR009W', # GAL3
                    'YPL248C', # GAL4
                    'YBR018C', # GAL7
                    'YBR019C', # GAL10
                    'YML051W', # GAL80
                    'YLR256W'] # HAP1

# exclude ergosterol pathway genes that are being used in GxE project
ergosterol_genes_file = "/home/users/rang/scratch/yeast/ergosterol/ergosterol_biosynthetic_process_genes.txt"
ergosterol_genes = pd.read_csv(ergosterol_genes_file, sep='\t')
ergosterol_genes = ergosterol_genes.rename(columns={'Gene Systematic Name':'gene_id', 'Gene':'gene_name'}).set_index('gene_id')

genes_to_exclude = genes_to_exclude + ergosterol_genes.index.tolist() 
num_of_interactions_df = num_of_interactions_df.query('~gene_id.isin(@genes_to_exclude)')

# find top genes with most genetic interactions
interaction_rich_genes = num_of_interactions_df.loc[num_of_interactions_df[['positive_interactions', 'negative_interactions']].sum(axis=1).sort_values(ascending=False).index].head(genes_cutoff)

# annotate genes by essentiality
essential_genes_file = working_dir+"S288C_essential_genes.txt"
essential_genes_df = pd.read_csv(essential_genes_file, sep='\t')
interaction_rich_genes.loc[:,'essential'] = interaction_rich_genes.index.isin(essential_genes_df['Gene Systematic Name'])

# annotate genes by paralog status - data from supp. table 2 in Byrne & Wolfe 2005
paralogs_file = working_dir+"S288C_paralogs.txt"
paralogs_df = pd.read_csv(paralogs_file, sep='\t', index_col='Byrne')
paralogs_ka_list = pd.concat([paralogs_df[['Gene 1', 'KA']].rename(columns={'Gene 1':'Gene'}), paralogs_df[['Gene 2', 'KA']].rename(columns={'Gene 2':'Gene'})], axis=0, ignore_index=True).set_index('Gene')
interaction_rich_genes.loc[:,'paralog'] = interaction_rich_genes.index.isin(paralogs_ka_list.index)
interaction_rich_genes.loc[interaction_rich_genes['paralog'],'KA'] = paralogs_ka_list.loc[interaction_rich_genes.index[interaction_rich_genes['paralog']], 'KA']

display(interaction_rich_genes.shape)
display(interaction_rich_genes.head())

(800, 11)

,positive_interactions,negative_interactions,chrom,start,end,strand,gene_name,description,essential,paralog,KA
gene_id,,,,,,,,,,,
YFL039C,1324,2220,chrVI,53260,54696,-,ACT1,"Actin, structural protein involved in cell pol...",True,False,NaN
YFL034C-B,1150,1897,chrVI,63016,63993,-,MOB2,"Component of the RAM signaling network, that a...",True,False,NaN
YJR076C,749,1342,chrX,575354,576601,-,CDC11,Component of the septin ring of the mother-bud...,True,False,NaN
YBL105C,841,1139,chrII,14241,17696,-,PKC1,Protein serine/threonine kinase essential for ...,True,False,NaN
YER157W,925,967,chrV,484788,487193,+,COG3,Essential component of the conserved oligomeri...,True,False,NaN


## Scan 1,011 genomes VCF for variants in and around gene of interest

In [11]:
# search hub genes for variants in each gvcf
chromosome_dict = {'chrI' : 'chromosome1',
                   'chrII': 'chromosome2',
                   'chrIII': 'chromosome3',
                   'chrIV': 'chromosome4',
                   'chrV': 'chromosome5',
                   'chrVI': 'chromosome6',
                   'chrVII': 'chromosome7',
                   'chrVIII': 'chromosome8',
                   'chrIX': 'chromosome9',
                   'chrX': 'chromosome10',
                   'chrXI': 'chromosome11',
                   'chrXII': 'chromosome12',
                   'chrXIII': 'chromosome13',
                   'chrXIV': 'chromosome14',
                   'chrXV': 'chromosome15',
                   'chrXVI': 'chromosome16',}

genes_to_process = []
for chrom in chromosome_dict.keys():
    genes_df = interaction_rich_genes.query('chrom==@chrom').sort_values('start')
    gvcf_file = "/home/users/rang/share/yeast/1011genomes/by_chrom/{}.gvcf.gz".format(chromosome_dict[chrom])
    genes_to_process.append((genes_df, gvcf_file, chromosome_dict))

# consolidate candidate variants
num_of_cores = min(len(os.sched_getaffinity(0)), len(genes_to_process))
with mp.Pool(num_of_cores) as pool:
    results = pool.starmap(select_variants, genes_to_process)
results = pd.concat(results).reset_index(drop=True)

# write to new vcf file
gxg_variants_file = "gxg_variants.vcf"
template = vcf.Reader(filename=genes_to_process[0][1])
# change chromosome naming convention to a single Roman numeral (follows ref genome, VEP formatting)
chromosome_dict_rev = {v:k for k, v in chromosome_dict.items()} 
template.contigs = OrderedDict((chromosome_dict_rev[k][3:], v._replace(id=chromosome_dict_rev[k][3:])) for k, v in template.contigs.items())
# remove 1011 genomes samples data
template.samples=[]

output = vcf.Writer(open(gxg_variants_file, 'w'), template)
for record in results:
    # update chromosome naming
    record.CHROM = chromosome_dict_rev[record.CHROM][3:]
    # write to file
    output.write_record(record)
output.close()

## Filter variants for those that can be targeted for REF to ALT allele editing
Use a 60 bp window to ensure the same sequence (ref allele +- 30bp) can be found in four yeast strains

In [12]:
# path to genome fasta files
ref_genome_fasta = "/home/users/rang/yeast/genomes/Saccharomyces_cerevisiae.R64-1-1.dna.chromosome.I.fa"
rm_genome_fasta = "/home/users/rang/yeast/genomes/RM11-1A_SGD_2015_JRIP00000000.fsa"
yjm_genome_fasta = "/home/users/rang/yeast/genomes/YJM789_Stanford_2007_AAFW02000000_highQuality31.fsa"
yps_genome_fasta = "/home/users/rang/yeast/genomes/YPS128.genome.fa"

# load genome files
ref_genome = SeqIO.to_dict(SeqIO.parse(ref_genome_fasta, "fasta"))
rm_genome = SeqIO.to_dict(SeqIO.parse(rm_genome_fasta, "fasta"))
yjm_genome = SeqIO.to_dict(SeqIO.parse(yjm_genome_fasta, "fasta"))
yps_genome = SeqIO.to_dict(SeqIO.parse(yps_genome_fasta, "fasta"))
genomes_list = [ref_genome, rm_genome, yjm_genome, yps_genome] # IMPORTANT: SET REFERENCE GENOME AS FIRST IN LIST

In [13]:
# path to input and output VCF files
gxg_variants_file = "gxg_variants.vcf"
gxg_variants_select_file = gxg_variants_file.replace('.vcf', '_select.vcf')

threshold = 10 # threshold value for searching NGGs
window = 61 # size of flanking sequences to search for unique hits

# read in all candidate variants to filter
gxg_variants = vcf.Reader(filename=gxg_variants_file)
gxg_variants = [rec for rec in gxg_variants]
# split variants into chunks for multiprocessing
n = len(gxg_variants)//len(os.sched_getaffinity(0)) + 1
gxg_variants = [gxg_variants[i:i+n] for i in range(0, len(gxg_variants), n)]

# find targetable variants (TGT info field indicates which genomes variant is targetable in)
to_process = [(rec_list, genomes_list, threshold, window) for rec_list in gxg_variants]
with mp.Pool(len(os.sched_getaffinity(0))) as pool:
    result = pool.starmap(find_targetable_variants, to_process) 

# write targetable variants to file
header = vcf.Reader(filename=gxg_variants_file)
# add "TGT" info field into VCF header
header_info = namedtuple('Info', ['id', 'num', 'type', 'desc', 'source', 'version'])
header_info_tgt = header_info('TGT', '.', 'Integer', 'Targetability of variant in tested genomes', None, None)
header.infos['TGT'] = header_info_tgt
# write to file
gxg_variants_select = vcf.Writer(open(gxg_variants_select_file, 'w'), header)
to_write = [rec for rec_list in result for rec in rec_list]
for rec in to_write:
    gxg_variants_select.write_record(rec)
gxg_variants_select.close()

## Filter variants by allele frequency, and editability in four strains
1. Exclude singletons and doubletons
2. Exclude variants that cannot be edited in all four strains
3. For variants with multiple alternate alleles, select most common alternate allele

Variants are assigned an ID prefix "GXG" to denote their use in the CRISPEY-BAR epistasis project

In [14]:
# candidate variants editable in all strains
gxg_variants_select = vcf.Reader(filename=gxg_variants_select_file)

# open output file
gxg_variants_select_AFfilter_file = gxg_variants_select_file.replace('.vcf', '_AFfilter.vcf')
gxg_variants_select_AFfilter = vcf.Writer(open(gxg_variants_select_AFfilter_file, 'w'), gxg_variants_select)

var_counter = {}
for record in gxg_variants_select:        
    # skip variant if all possible alternate allele count is less than 3
    if all([ac<3 for ac in record.INFO['AC']]):
        continue
    # if at least one alternate allele meets minimum count, select most common alternate allele to study
    else:
        i = np.argmax(record.INFO['AC']) # most common alt allele
        # update record entry
        record.ALT = [record.ALT[i]]
        record.INFO['AC'] = [record.INFO['AC'][i]]
        record.INFO['AF'] = [record.INFO['AF'][i]]
        record.INFO['MLEAC'] = [record.INFO['MLEAC'][i]]
        record.INFO['MLEAF'] = [record.INFO['MLEAF'][i]]
        
        # inspect TGT field and assign variant ID prefix
        ### if tgt field contains all 1 (i.e. targetable by all queried genomes), assign "GXG" prefix to variant ID
        ### otherwise, skip variant
        if all([i==1 for i in record.INFO['TGT']]):
            var_id_prefix = "GXG"
        else:
#             print('variant does not pass TGT check: skipping')
            continue
            
        # add to counter
        if var_id_prefix not in var_counter:
            var_counter[var_id_prefix] = 1
        else:
            var_counter[var_id_prefix] += 1
        
        # add unique variant ID (follow format XXX_XXXXX)
        record.ID='{}_{:05d}'.format(var_id_prefix, var_counter[var_id_prefix])
        
        # write to output
        gxg_variants_select_AFfilter.write_record(record)
gxg_variants_select_AFfilter.close()

## Submit gxg_variants_select_AFfilter VCF to VEP to get annotations, and download VEP output in TXT format (vep_output_filename).
http://uswest.ensembl.org/Saccharomyces_cerevisiae/Tools/VEP

In [15]:
# add annotations to variants 
gff_file = '/home/users/rang/yeast/genomes/saccharomyces_cerevisiae_R64-1-1_20110208.gff'
gxg_variants_file = "gxg_variants.vcf"
gxg_variants_select_file = gxg_variants_file.replace('.vcf', '_select.vcf')
gxg_variants_select_AFfilter_file = gxg_variants_select_file.replace('.vcf', '_AFfilter.vcf')
vep_output_filename = gxg_variants_select_AFfilter_file.replace('.vcf', '_VEPoutput.txt')

variants_annotated_filename = vep_output_filename.replace('_VEPoutput.txt', '_annotated.txt')
variants_annotated_df = annotate_variants_by_VEPoutput(gxg_variants_select_AFfilter_file, 
                                                       vep_output_filename, 
                                                       gff_file, 
                                                       variants_annotated_filename, 
                                                       id_colname='var_id')

parsing entry: 5000
parsing entry: 10000
parsing entry: 15000
parsing entry: 20000
parsing entry: 25000
parsing entry: 30000
parsing entry: 35000
parsing entry: 40000
parsing entry: 45000
parsing entry: 50000


In [16]:
# read in annotated variant file
variants_annotated_df = pd.read_csv(variants_annotated_filename, sep='\t', index_col=0)

# remove variants from genes not listed in interaction_rich_genes
intergenic_regions_list = ['bidirectional_promoter', 'unidirectional_promoter', 'intergenic']
gxg_design_oligos = variants_annotated_df.query('(Gene_ID.isin(@interaction_rich_genes.index) | region.isin(@intergenic_regions_list))').copy()
display(gxg_design_oligos.shape)
display(gxg_design_oligos.head())

(44835, 41)

,CHROM,POS,REF,ALT,AC,AN,AF,upstream_distance_str,downstream_distance_str,region,...,Codons,DOMAINS,closest_gene1_Gene_Name,closest_gene1_Gene_ID,closest_gene1_Annotation,closest_gene1_Distance,closest_gene2_Gene_Name,closest_gene2_Gene_ID,closest_gene2_Annotation,closest_gene2_Distance
var_id,,,,,,,,,,,,,,,,,,,,,
GXG_00005,I,42019,A,C,61,2022,0.030000,3880|158,862|118|3047|4872,unidirectional_promoter,...,NaN,NaN,GPB2,YAL056W,downstream_gene_variant,118.0,PEX22,YAL055W,upstream_gene_variant,158.0
GXG_00006,I,42023,C,T,12,2020,0.005941,3876|154,858|122|3051|4876,unidirectional_promoter,...,NaN,NaN,GPB2,YAL056W,downstream_gene_variant,122.0,PEX22,YAL055W,upstream_gene_variant,154.0
GXG_00007,I,42033,A,AT,60,2012,0.030000,3865|143,847|132|3061|4886,unidirectional_promoter,...,NaN,NaN,GPB2,YAL056W,downstream_gene_variant,132.0,PEX22,YAL055W,upstream_gene_variant,143.0
GXG_00008,I,42036,TTGG,CTGG,6,2016,0.002976,3860|138,842|135|3064|4889,unidirectional_promoter,...,NaN,NaN,GPB2,YAL056W,downstream_gene_variant,135.0,PEX22,YAL055W,upstream_gene_variant,138.0
GXG_00009,I,42037,TGG,T,30,2022,0.015000,3860|138,842|137|3066|4891,unidirectional_promoter,...,NaN,NaN,GPB2,YAL056W,downstream_gene_variant,137.0,PEX22,YAL055W,upstream_gene_variant,138.0


In [17]:
# write variants to gxg_design_oligos VCF as a preliminary list to design oligos for
gxg_design_oligos_vcf_file = gxg_variants_file.replace('.vcf', '_design_oligos_initial.vcf')
gxg_variants_select_AFfilter = vcf.Reader(filename=gxg_variants_select_AFfilter_file)
gxg_design_oligos_vcf = vcf.Writer(open(gxg_design_oligos_vcf_file, 'w'), gxg_variants_select_AFfilter)
for record in gxg_variants_select_AFfilter:
    if record.ID in gxg_design_oligos.index:
        gxg_design_oligos_vcf.write_record(record)
gxg_design_oligos_vcf.close()

## Run oligo design script (crispey3_design_library.ipynb) with these variants to identify which can be targeted by at least two unique guides

In [18]:
gxg_oligo_file = os.path.expanduser("~/crispey3/initial_design/Output/all_SNPs_gxg_initial_GG_9bp_OLIGO.tab")
gxg_oligo_df = pd.read_csv(gxg_oligo_file, sep='\t')

gxg_design_oligos = pd.read_csv("/home/users/rang/scratch/yeast/genetic_interactions/costanzo_2016/gxg_variants_select_AFfilter_annotated.txt", sep='\t', index_col=0)
# remove variants that fall within genes, but are not in the approved genes list
intergenic_regions_list = ['bidirectional_promoter', 'unidirectional_promoter', 'intergenic']
gxg_design_oligos = gxg_design_oligos.query('(Gene_ID.isin(@interaction_rich_genes.index) | region.isin(@intergenic_regions_list))').copy()

# select oligos that have at least 2 oligos designed
gxg_design_oligos['num_of_oligos'] = gxg_oligo_df.groupby('var_id').apply(len)
gxg_oligos_final = gxg_design_oligos.query('num_of_oligos>1').copy()
display(gxg_oligos_final.shape)
display(gxg_oligos_final.head())

(18255, 42)

,CHROM,POS,REF,ALT,AC,AN,AF,upstream_distance_str,downstream_distance_str,region,...,DOMAINS,closest_gene1_Gene_Name,closest_gene1_Gene_ID,closest_gene1_Annotation,closest_gene1_Distance,closest_gene2_Gene_Name,closest_gene2_Gene_ID,closest_gene2_Annotation,closest_gene2_Distance,num_of_oligos
var_id,,,,,,,,,,,,,,,,,,,,,
GXG_00006,I,42023,C,T,12,2020,0.005941,3876|154,858|122|3051|4876,unidirectional_promoter,...,NaN,GPB2,YAL056W,downstream_gene_variant,122.0,PEX22,YAL055W,upstream_gene_variant,154.0,2.0
GXG_00007,I,42033,A,AT,60,2012,0.030000,3865|143,847|132|3061|4886,unidirectional_promoter,...,NaN,GPB2,YAL056W,downstream_gene_variant,132.0,PEX22,YAL055W,upstream_gene_variant,143.0,2.0
GXG_00012,I,42184,C,T,5,2022,0.002473,3715,697|283|3212,missense_variant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
GXG_00013,I,42186,C,T,59,2022,0.029000,3713,695|285|3214,missense_variant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
GXG_00014,I,42452,C,T,8,2022,0.003956,3447,429|551|3480,synonymous_variant,...,"Pfam:PF12827,Gene3D:3.40.50.11730",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0


## Of the 18,255 natural variants eligible for the CRISPEY-BAR epistasis library, select ~2,000 variants - yields a final library of ~5k oligos
1. Two groups of 1,000 variants are selected. Group 1 consists of variants with allele frequency (AF) greater than or equal to 2%. Group 2 consists of "ultra-rare" variants with allele frequency less than or equal to 0.2%
2. Genes are ranked, in descending order, by the number of variants with AF>=2%, followed by number of variants with AF<=0.2%.
3. For each gene, select all available group 1 variants, then select a matching number of group 2 variants.
4. Go through each gene and select variants till at least 2,000 variants are selected for library.

In [19]:
# create a dataframe of hub genes with annotations and number of variants in each group
left_limit=500 # distance from left of gene to search
right_limit=500 # distance from right of gene to search

genes_df = interaction_rich_genes.copy()
genes_df.loc[:,'chrom'] = genes_df['chrom'].str.replace('chr', '')
genes_df.loc[:,'left_limit'] = genes_df['start'] - left_limit
genes_df.loc[:,'right_limit'] = genes_df['end'] + right_limit
genes_df.loc[:, 'region_length'] = genes_df['right_limit'] - genes_df['left_limit'] + 1

# associate each variant with hub gene of interest if it falls within gene limits
gxg_oligos_final.loc[:,'assoc_gene'] = gxg_oligos_final.apply(lambda x: genes_df.query('chrom==@x.CHROM & left_limit<=@x.POS & right_limit>=@x.POS').index.tolist(), axis=1)
# remove variants without gene associations
gxg_oligos_final = gxg_oligos_final[gxg_oligos_final.assoc_gene.apply(len)>0]
# for coding variants with multiple associations (i.e. genes of interest adjacent to each other),
# adjust to associate only gene whose coding region contains variant
selection = (gxg_oligos_final.assoc_gene.apply(len)>1) & (~gxg_oligos_final.region.isin(intergenic_regions_list))
gxg_oligos_final.loc[selection, 'assoc_gene'] = gxg_oligos_final.loc[selection, 'Gene_ID'].apply(lambda x: [x])

# # finally, exclude remaining variants that are associated with more than one gene (should be only noncoding variants)
# gxg_oligos_final = gxg_oligos_final[gxg_oligos_final['assoc_gene'].apply(lambda x: len(x)==1)]

# count the number of variants associated with each gene
af_cutoff_common=0.02
af_cutoff_rare=0.002
genes_df.loc[:, 'num_of_vars'] = [gxg_oligos_final['assoc_gene'].apply(lambda x: gene in x).sum() for gene in genes_df.index]
genes_df.loc[:, 'num_of_vars_common'] = [gxg_oligos_final.query('AF>=@af_cutoff_common')['assoc_gene'].apply(lambda x: gene in x).sum() for gene in genes_df.index]
genes_df.loc[:, 'num_of_vars_rare'] = [gxg_oligos_final.query('AF<=@af_cutoff_rare')['assoc_gene'].apply(lambda x: gene in x).sum() for gene in genes_df.index]
# estimate density of variants (num of variants per 100bp) surveyed in each gene_region
genes_df.loc[:, 'var_density'] = genes_df['num_of_vars'] / genes_df['region_length'] * 100

genes_df = genes_df.sort_values(['num_of_vars_common', 'num_of_vars_rare'], ascending=False)
display(genes_df.shape)
display(genes_df.head())

(800, 18)

,positive_interactions,negative_interactions,chrom,start,end,strand,gene_name,description,essential,paralog,KA,left_limit,right_limit,region_length,num_of_vars,num_of_vars_common,num_of_vars_rare,var_density
gene_id,,,,,,,,,,,,,,,,,,
YNL262W,252,319,XIV,148212,154880,+,POL2,Catalytic subunit of DNA polymerase (II) epsil...,True,False,NaN,147712,155380,7669,99,24,15,1.290911
YPL231W,348,548,XVI,108652,114315,+,FAS2,"Alpha subunit of fatty acid synthetase, which ...",True,False,NaN,108152,114815,6664,99,21,21,1.485594
YMR207C,242,298,XIII,677193,683564,-,HFA1,"Mitochondrial acetyl-coenzyme A carboxylase, c...",False,True,0.366,676693,684064,7372,81,19,22,1.098752
YGR097W,159,370,VII,678695,682135,+,ASK10,"Component of RNA polymerase II holoenzyme, pho...",False,True,0.561,678195,682635,4441,52,19,12,1.170907
YFL033C,276,396,VI,69115,74427,-,RIM15,Glucose-repressible protein kinase involved in...,False,False,NaN,68615,74927,6313,61,18,12,0.966260


In [20]:
# select 2000 variants, in as few genes as possible
cutoff = 2000

var_list = []
gene_list = []
exclude = []
for gene in genes_df.index:
    # skip blacklisted genes in exclusion list
    if gene in exclude:
#         print(gene, 'is excluded')
        continue
        
    df = gxg_oligos_final[gxg_oligos_final['assoc_gene'].apply(lambda x: gene in x)].sort_values(['AF', 'AC'], ascending=False)
    com_var = df.query('AF>=@af_cutoff_common').index.tolist()
    if len(com_var) > len(df)//2:
        com_var = com_var[:len(df)//2]        
    rar_var = df.tail(len(com_var)).index.tolist()
    com_rar_var = com_var + rar_var
    
    # if selected variants are associated with genes other than the one being parsed (e.g. two hub genes adjacent to each other),
    # add those other genes to the exclude list to avoid resampling the same variants
    other_genes = set([g for l in df.loc[com_rar_var, 'assoc_gene'] for g in l if g != gene])
    if len(other_genes)>0:
#         print(other_genes, "added to exclude list")
        exclude += list(other_genes)
    
    # add selected variants to var_list
    var_list += com_rar_var
    # add gene to gene_list
    gene_list.append(gene)
    
    if len(var_list)>= cutoff:
        break

# number of genes selected
hub_genes = genes_df.loc[gene_list]
print('Number of hub genes selected:', len(hub_genes))

# select final list of variants that will be targeted
gxg_oligos_final = gxg_oligos_final.query('var_id.isin(@var_list)')
display(gxg_oligos_final.shape)
display(gxg_oligos_final.head())

# write gxg_oligos_final table to file
gxg_oligos_final.to_csv("/home/users/rang/scratch/yeast/genetic_interactions/costanzo_2016/gxg_variants_final.txt", sep='\t')

Number of hub genes selected: 103


(2000, 43)

,CHROM,POS,REF,ALT,AC,AN,AF,upstream_distance_str,downstream_distance_str,region,...,closest_gene1_Gene_Name,closest_gene1_Gene_ID,closest_gene1_Annotation,closest_gene1_Distance,closest_gene2_Gene_Name,closest_gene2_Gene_ID,closest_gene2_Annotation,closest_gene2_Distance,num_of_oligos,assoc_gene
var_id,,,,,,,,,,,,,,,,,,,,,
GXG_00321,I,101530,G,C,4,2022,0.001978,385|1833,1662|4742|35,unidirectional_promoter,...,MAK16,YAL025C,upstream_gene_variant,385.0,LTE1,YAL024C,downstream_gene_variant,35.0,3.0,"[YAL024C, YAL025C]"
GXG_00329,I,101816,C,T,4,2020,0.001980,671|2119,1948|4456,missense_variant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,[YAL024C]
GXG_00332,I,101901,C,A,90,2022,0.045000,756|2204,2033|4371,synonymous_variant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,[YAL024C]
GXG_00339,I,102261,C,T,78,2020,0.039000,1116|2564,2393|4011,synonymous_variant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,[YAL024C]
GXG_00346,I,102699,G,A,310,2016,0.154000,1554|3002,2831|3573,synonymous_variant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,[YAL024C]


## Summary of GXG variants

In [21]:
print("Number of variants:", len(gxg_oligos_final))
print("Number of oligos:", int(sum(gxg_oligos_final.num_of_oligos)))
print()
print("Distribution of (common) variants among hub genes")
# print(gxg_oligos_final.Gene_ID.value_counts().filter(hub_genes.index).describe())
print(hub_genes.num_of_vars_common.describe())
print()
print("Distribution of oligos among GxG variants")
print(gxg_oligos_final.num_of_oligos.describe())
print()
print("Proportion of variant annotation classes")
print(gxg_oligos_final.region.value_counts()/len(gxg_oligos_final))

Number of variants: 2000
Number of oligos: 4957

Distribution of (common) variants among hub genes
count    103.000000
mean       9.708738
std        3.285831
min        7.000000
25%        8.000000
50%        9.000000
75%       11.000000
max       24.000000
Name: num_of_vars_common, dtype: float64

Distribution of oligos among GxG variants
count    2000.000000
mean        2.478500
std         0.740143
min         2.000000
25%         2.000000
50%         2.000000
75%         3.000000
max         7.000000
Name: num_of_oligos, dtype: float64

Proportion of variant annotation classes
synonymous_variant         0.4395
missense_variant           0.3470
unidirectional_promoter    0.1020
bidirectional_promoter     0.0835
intergenic                 0.0175
intron_variant             0.0030
inframe_insertion          0.0030
inframe_deletion           0.0025
frameshift_variant         0.0010
stop_gained                0.0010
Name: region, dtype: float64


## Determine number of wells required in matrixed oligo synthesis array
Twist Biosciences synthesizes 121 oligos per well, up to 384 wells per plate.

In [22]:
# count OLIGOS in each variant prefix set
display(gxg_oligos_final.groupby(lambda x: x[:3]).num_of_oligos.sum().sort_values(ascending=False))

# determine number of pools to use
num_of_oligos = int(gxg_oligos_final.num_of_oligos.sum())
min_pool_size = 109 # allows 12 technical oligos
max_pool_size = 118 # allows 3 technical oligos
complete_pool_size = 121 # maximum oligos synthesized per pool

# find number of pools to use
find_pool_size_options(num_of_oligos, min_pool_size, max_pool_size, complete_pool_size)

GXG    4957.0
Name: num_of_oligos, dtype: float64

Total number of oligos to fit: 4957


(42, 45)

Use 42 pools:
Number of oligos per pool: 118
Number of oligos leftover: 1
Number of technical oligos: 126

Use 43 pools:
Number of oligos per pool: 115
Number of oligos leftover: 12
Number of technical oligos: 258

Use 44 pools:
Number of oligos per pool: 112
Number of oligos leftover: 29
Number of technical oligos: 396

Use 45 pools:
Number of oligos per pool: 110
Number of oligos leftover: 7
Number of technical oligos: 495



array([42, 43, 44, 45])

# Selecting natural variants in yeast QTLs for GxE project
Screen natural variants in select list of QTLs from Bloom et al. (2019) tested for environment-specific fitness effects.

In [23]:
def select_variants_from_qtls(qtls_df, vcf_file, chrom_dict, upstream_length=500, downstream_length=500):
    '''
    modified version of select_variants function that accepts a dataframe of QTLs
    (requires qtl_left and qtl_right columns defined, and assumes each QTL is associated with one gene)
    defines search boundaries for each QTL and then searches vcf for variants within it
    can be parallelized by splitting genes and vcf by chromosome
    '''
    # find gene upstream and downstream boundaries
    qtls_df.loc[qtls_df['strand']=='+', 'gene_left'] = qtls_df['start']-upstream_length
    qtls_df.loc[qtls_df['strand']=='+', 'gene_right'] = qtls_df['end']+downstream_length
    qtls_df.loc[qtls_df['strand']=='-', 'gene_left'] = qtls_df['start']-downstream_length
    qtls_df.loc[qtls_df['strand']=='-', 'gene_right'] = qtls_df['end']+upstream_length
    
    # define search boundaries by either gene or QTL boundary, whichever is larger
    qtls_df.loc[:,'search_left'] = qtls_df[['gene_left', 'qtl_left']].min(axis=1)
    qtls_df.loc[:,'search_right'] = qtls_df[['gene_right', 'qtl_right']].max(axis=1)
    qtls_df.loc[:,'search_left'] = qtls_df['search_left'].astype(int)
    qtls_df.loc[:,'search_right'] = qtls_df['search_right'].astype(int)
    
    # from these search boundaries, remove duplicates
    search_boundaries_df = qtls_df[['chrom', 'search_left', 'search_right']].drop_duplicates()

    # read in vcf file to parse for variants
    allVars = vcf.Reader(filename=vcf_file)
    candVars = []
    for i, row in search_boundaries_df.iterrows():
        retrieve = allVars.fetch(chrom=chrom_dict[row['chrom']], 
                                 start=row['search_left']-1,
                                 end=row['search_right'])
        for rec in retrieve:
            rec.samples = []
            rec.FORMAT=None
            candVars.append(rec)
    candVars = pd.Series(candVars).drop_duplicates().reset_index(drop=True)
    return candVars

In [24]:
# set working directory
working_dir = "/home/users/rang/scratch/yeast/GxE/"
# list of QTLs to parse for GxE variants
gxe_qtls_file = working_dir+"GxE_gene_selection_20201130.tsv"
os.chdir(working_dir)

In [25]:
# read in GxE QTL table
gxe_qtls = pd.read_csv(gxe_qtls_file, sep='\t', index_col=0)
gxe_qtls = gxe_qtls.rename(columns={'Systematic Name':'gene_id', 'Gene ID':'gene_name', 'Chr':'chrom'})#.set_index('gene_id')
# add gene strand and description annotations to gxe QTLs
gxe_qtls = gxe_qtls.merge(gene_info_df[['strand', 'description']], how='left', left_on='gene_id', right_index=True)

# remove QTLs containing blacklisted genes
genes_to_exclude = ['YDL227C', # HO
                    'YOR202W', # HIS3
                    'YEL021W', # URA3
                    'YCL018W', # LEU2
                    'YBR115C', # LYS2
                    'YBR020W', # GAL1
                    'YDR009W', # GAL3
                    'YPL248C', # GAL4
                    'YBR018C', # GAL7
                    'YBR019C', # GAL10
                    'YML051W', # GAL80
                    'YLR256W'] # HAP1
                   
gxe_qtls = gxe_qtls.query('~gene_id.isin(@genes_to_exclude)')

# get list of 1,011 yeast strains used in QTL mapping
yeast_strains_id = gxe_qtls['Strain ID of Parent 1 in Peter et al. 2018'].dropna().unique()
yeast_strains_name_to_id = dict(zip(gxe_qtls['Parent 1'].apply(lambda x: x.split(' ')[0]),
                                    gxe_qtls['Strain ID of Parent 1 in Peter et al. 2018']))
display(gxe_qtls.shape)
display(gxe_qtls.head())

(92, 33)

,trait,chr,pos,cross,forward_scan_marker,index,p-value,q-value,LOD,relocalized_peak,...,Beta_abs,qtl_genes,num_qtl_genes,gene_id,gene_name,chrom,start,end,strand,description
0,Caffeine;15mM;2,chrXV,624258,B,chrXV_620334_A_G_61508,18833,0.0001,0.0001,74.281303,chrXV_624258_G_A_61750,...,0.909699,['YOR153W'],1,YOR153W,PDR5,chrXV,619840,624375,+,Plasma membrane ATP-binding cassette (ABC) tra...
1,Fluconazole;100uM;2,chrXV,621444,B,chrXV_621444_A_G_61540,18839,0.0001,0.0001,117.931452,chrXV_621444_A_G_61540,...,0.920556,['YOR153W'],1,YOR153W,PDR5,chrXV,619840,624375,+,Plasma membrane ATP-binding cassette (ABC) tra...
2,Fluconazole;100uM;2,chrXV,619941,377,chrXV_619941_G_T_47212,23734,0.0001,0.0001,168.252715,chrXV_619941_G_T_47212,...,0.711397,['YOR153W'],1,YOR153W,PDR5,chrXV,619840,624375,+,Plasma membrane ATP-binding cassette (ABC) tra...
3,Fluconazole;100uM;2,chrXV,622413,3049,chrXV_622413_A_G_72894,25133,0.0001,0.0001,120.009983,chrXV_622413_A_G_72894,...,0.607320,['YOR153W'],1,YOR153W,PDR5,chrXV,619840,624375,+,Plasma membrane ATP-binding cassette (ABC) tra...
4,Fluconazole;100uM;2,chrXV,624320,377,chrXV_474477_C_A_46484,23322,0.0009,0.0003,7.932991,chrXV_624320_TTTTC_T_47455,...,0.107987,['YOR153W'],1,YOR153W,PDR5,chrXV,619840,624375,+,Plasma membrane ATP-binding cassette (ABC) tra...


## Scan 1,011 genomes VCF for variants in and around QTLs of interest

In [26]:
# search GxE QTLs for variants in each gvcf
chromosome_dict = {'chrI' : 'chromosome1',
                   'chrII': 'chromosome2',
                   'chrIII': 'chromosome3',
                   'chrIV': 'chromosome4',
                   'chrV': 'chromosome5',
                   'chrVI': 'chromosome6',
                   'chrVII': 'chromosome7',
                   'chrVIII': 'chromosome8',
                   'chrIX': 'chromosome9',
                   'chrX': 'chromosome10',
                   'chrXI': 'chromosome11',
                   'chrXII': 'chromosome12',
                   'chrXIII': 'chromosome13',
                   'chrXIV': 'chromosome14',
                   'chrXV': 'chromosome15',
                   'chrXVI': 'chromosome16'}

qtls_to_process = []
for chrom in chromosome_dict.keys():
    qtls_df = gxe_qtls.query('chrom==@chrom').sort_values('pos')
    gvcf_file = "/home/users/rang/share/yeast/1011genomes/by_chrom/{}.gvcf.gz".format(chromosome_dict[chrom])
    qtls_to_process.append((qtls_df, gvcf_file, chromosome_dict))
    
# consolidate candidate variants
num_of_cores = min(len(os.sched_getaffinity(0)), len(qtls_to_process))
with mp.Pool(num_of_cores) as pool:
    results = pool.starmap(select_variants_from_qtls, qtls_to_process)
results = pd.concat(results).reset_index(drop=True)

# write to new vcf file
gxe_variants_file = "gxe_variants.vcf"
template = vcf.Reader(filename=qtls_to_process[0][1])
# change chromosome naming convention to a single Roman numeral (follows ref genome, VEP formatting)
chromosome_dict_rev = {v:k for k, v in chromosome_dict.items()} 
template.contigs = OrderedDict((chromosome_dict_rev[k][3:], v._replace(id=chromosome_dict_rev[k][3:])) for k, v in template.contigs.items())
# remove 1011 genomes strain allele info
template.samples = []

output = vcf.Writer(open(gxe_variants_file, 'w'), template)
for record in results:
    # update chromosome naming
    record.CHROM = chromosome_dict_rev[record.CHROM][3:]
    # write to file
    output.write_record(record)
output.close()

## Filter variants for those that can be targeted for editing

In [27]:
# path to genome fasta files
ref_genome_fasta = "/home/users/rang/yeast/genomes/Saccharomyces_cerevisiae.R64-1-1.dna.chromosome.I.fa"

# load genome files
ref_genome = SeqIO.to_dict(SeqIO.parse(ref_genome_fasta, "fasta"))
genomes_list = [ref_genome]

In [28]:
# path to input and output VCF files
gxe_variants_file = "gxe_variants.vcf"
gxe_variants_select_file = gxe_variants_file.replace('.vcf', '_select.vcf')

threshold = 10 # threshold value for searching NGGs
window = 61 # size of flanking sequences to search for unique hits

# read in all candidate variants to filter
gxe_variants = vcf.Reader(filename=gxe_variants_file)
gxe_variants = [rec for rec in gxe_variants]
# split variants into chunks for multiprocessing
n = len(gxe_variants)//len(os.sched_getaffinity(0)) + 1
gxe_variants = [gxe_variants[i:i+n] for i in range(0, len(gxe_variants), n)]

# find targetable variants (TGT info field indicates which genomes variant is targetable in)
to_process = [(rec_list, genomes_list, threshold, window) for rec_list in gxe_variants]
with mp.Pool(len(os.sched_getaffinity(0))) as pool:
    result = pool.starmap(find_targetable_variants, to_process) 

# write targetable variants to file
header = vcf.Reader(filename=gxe_variants_file)
# add "TGT" info field into VCF header
header_info = namedtuple('Info', ['id', 'num', 'type', 'desc', 'source', 'version'])
header_info_tgt = header_info('TGT', '.', 'Integer', 'Targetability of variant in tested genomes', None, None)
header.infos['TGT'] = header_info_tgt
# write to file
gxe_variants_select = vcf.Writer(open(gxe_variants_select_file, 'w'), header)
to_write = [rec for rec_list in result for rec in rec_list]
for rec in to_write:
    gxe_variants_select.write_record(rec)
gxe_variants_select.close()

## Filter variants by allele frequency
1. For variants with multiple alternate alleles, split variant entry into separate lines in VCF for library design - allows oligo design pipeline to synthesize separate oligos for each type of REF-to-ALT edit

Variants are assigned an ID prefix "GXE" to denote their use in the CRISPEY-BAR GxE project

In [29]:
# select variants by allele frequency (currently disabled singleton/doubleton filter)
### if multiple alternate alleles, SPLIT VARIANT ENTRY INTO SEPARATE ENTRIES in VCF (useful for oligo design pipeline)
### assign unitl variant ID prefix and number (use format VAR_XXXXX)

# candidate variants editable in all strains
gxe_variants_select = vcf.Reader(filename=gxe_variants_select_file)

# open output file
gxe_variants_select_AFfilter_file = gxe_variants_select_file.replace('.vcf', '_AFfilter.vcf')
gxe_variants_select_AFfilter = vcf.Writer(open(gxe_variants_select_AFfilter_file, 'w'), gxe_variants_select)

var_counter = {}
for record in gxe_variants_select:        
    # (optional step to skip singletons and doubletons -- DISABLED by setting ac<1)
    if all([ac<1 for ac in record.INFO['AC']]):
        continue
    # if at least one alternate allele meets minimum count, write to VCF
    else:
        alt = record.ALT[:]
        ac = record.INFO['AC'][:]
        af = record.INFO['AF'][:]
        mleac = record.INFO['MLEAC'][:]
        mleaf = record.INFO['MLEAF'][:]
        
        # split each alt allele into its own variant entry (useful if a locus has multiple possible alleles)
        for i in range(len(alt)):
            record.ALT = [alt[i]]
            record.INFO['AC'] = [ac[i]]
            record.INFO['AF'] = [af[i]]
            record.INFO['MLEAC'] = [mleac[i]]
            record.INFO['MLEAF'] = [mleaf[i]]

            # inspect TGT field and assign variant ID prefix
            ### if tgt field contains all 1 (i.e. targetable by all queried genomes), assign "GXE"
            ### otherwise, skip variant
            if all([i==1 for i in record.INFO['TGT']]):
                var_id_prefix = "GXE"
            else:
#                 print('variant does not pass TGT check: skipping')
                continue

            # add to counter
            if var_id_prefix not in var_counter:
                var_counter[var_id_prefix] = 1
            else:
                var_counter[var_id_prefix] += 1

            # add unique variant ID (follow format VAR_XXXXX)
            record.ID='{}_{:05d}'.format(var_id_prefix, var_counter[var_id_prefix])

            # write to output
            gxe_variants_select_AFfilter.write_record(record)
            
gxe_variants_select_AFfilter.close()

## Submit gxe_variants_select_AFfilter VCF to VEP to get annotations, and download VEP output in TXT format (vep_output_filename).
http://uswest.ensembl.org/Saccharomyces_cerevisiae/Tools/VEP

In [30]:
# add annotations to variants 
gff_file = '/home/users/rang/yeast/genomes/saccharomyces_cerevisiae_R64-1-1_20110208.gff'
gxe_variants_file = "gxe_variants.vcf"
gxe_variants_select_file = gxe_variants_file.replace('.vcf', '_select.vcf')
gxe_variants_select_AFfilter_file = gxe_variants_select_file.replace('.vcf', '_AFfilter.vcf')
vep_output_filename = gxe_variants_select_AFfilter_file.replace('.vcf', '_VEPoutput.txt')

variants_annotated_filename = vep_output_filename.replace('_VEPoutput.txt', '_annotated.txt')
variants_annotated_df = annotate_variants_by_VEPoutput(gxe_variants_select_AFfilter_file, 
                                                       vep_output_filename, 
                                                       gff_file, 
                                                       variants_annotated_filename, 
                                                       id_colname='var_id')

parsing entry: 5000
parsing entry: 10000
parsing entry: 15000


In [31]:
# read in annotated file
variants_annotated_df = pd.read_csv(variants_annotated_filename, sep='\t', index_col=0)

# remove variants that fall within genes, but are not in the approved genes list
intergenic_regions_list = ['bidirectional_promoter', 'unidirectional_promoter', 'intergenic']
gxe_design_oligos = variants_annotated_df.query('(Gene_ID.isin(@gxe_qtls.gene_id.unique()) | region.isin(@intergenic_regions_list))').copy()
display(gxe_design_oligos.shape)
display(gxe_design_oligos.head())

(15390, 41)

,CHROM,POS,REF,ALT,AC,AN,AF,upstream_distance_str,downstream_distance_str,region,...,Codons,DOMAINS,closest_gene1_Gene_Name,closest_gene1_Gene_ID,closest_gene1_Annotation,closest_gene1_Distance,closest_gene2_Gene_Name,closest_gene2_Gene_ID,closest_gene2_Annotation,closest_gene2_Distance
var_id,,,,,,,,,,,,,,,,,,,,,
GXE_00017,I,192257,G,A,8,2006,0.003988,362,4525|3446|80|1,intergenic,...,NaN,NaN,YAT1,YAR035W,downstream_gene_variant,1.0,NaN,YAR035C-A,downstream_gene_variant,80.0
GXE_00023,I,192419,C,A,6,2020,0.002970,2|200,4687|3608|163,bidirectional_promoter,...,NaN,NaN,NaN,YAR035C-A,upstream_gene_variant,2.0,SWH1,YAR042W,upstream_gene_variant,200.0
GXE_00024,I,192424,GTTTGGATTACCTCT,ATTTGGATTACCTCT,39,2014,0.019000,7|181,4692|3613|168,bidirectional_promoter,...,NaN,NaN,NaN,YAR035C-A,upstream_gene_variant,7.0,SWH1,YAR042W,upstream_gene_variant,181.0
GXE_00025,I,192424,GTTTGGATTACCTCT,G,2,2014,0.000993,8|181,4693|3614|169,bidirectional_promoter,...,NaN,NaN,NaN,YAR035C-A,upstream_gene_variant,8.0,SWH1,YAR042W,upstream_gene_variant,181.0
GXE_00026,I,192427,T,C,902,2000,0.451000,10|192,4695|3616|171,bidirectional_promoter,...,NaN,NaN,NaN,YAR035C-A,upstream_gene_variant,10.0,SWH1,YAR042W,upstream_gene_variant,192.0


In [32]:
# write variants to gxe_design_oligos VCF as a preliminary list to design oligos for
gxe_design_oligos_vcf_file = gxe_variants_file.replace('.vcf', '_design_oligos_initial.vcf')

gxe_variants_select_AFfilter = vcf.Reader(filename=gxe_variants_select_AFfilter_file)
gxe_design_oligos_vcf = vcf.Writer(open(gxe_design_oligos_vcf_file, 'w'), gxe_variants_select_AFfilter)
for record in gxe_variants_select_AFfilter:
    if record.ID in gxe_design_oligos.index:
        gxe_design_oligos_vcf.write_record(record)
gxe_design_oligos_vcf.close()

## Run oligo design script (crispey3_design_library.ipynb) with these variants to identify which can be targeted by at least two unique guides

In [33]:
gxe_oligo_file = os.path.expanduser("~/crispey3/initial_design/Output/all_SNPs_gxe_initial_GG_9bp_OLIGO.tab")
gxe_oligo_df = pd.read_csv(gxe_oligo_file, sep='\t')

gxe_design_oligos = pd.read_csv("/home/users/rang/scratch/yeast/GxE/gxe_variants_select_AFfilter_annotated.txt", sep='\t', index_col=0)
# remove variants that fall within genes, but are not in the approved genes list
intergenic_regions_list = ['bidirectional_promoter', 'unidirectional_promoter', 'intergenic']
gxe_design_oligos = gxe_design_oligos.query('(Gene_ID.isin(@gxe_qtls.gene_id.unique()) | region.isin(@intergenic_regions_list))').copy()

# select oligos that have at least 2 oligos designed
gxe_design_oligos['num_of_oligos'] = gxe_oligo_df.groupby('var_id').apply(len)
gxe_oligos_final = gxe_design_oligos.query('num_of_oligos>1').copy()
display(gxe_oligos_final.shape)
display(gxe_oligos_final.head())

(6480, 42)

,CHROM,POS,REF,ALT,AC,AN,AF,upstream_distance_str,downstream_distance_str,region,...,DOMAINS,closest_gene1_Gene_Name,closest_gene1_Gene_ID,closest_gene1_Annotation,closest_gene1_Distance,closest_gene2_Gene_Name,closest_gene2_Gene_ID,closest_gene2_Annotation,closest_gene2_Distance,num_of_oligos
var_id,,,,,,,,,,,,,,,,,,,,,
GXE_00023,I,192419,C,A,6,2020,0.002970,2|200,4687|3608|163,bidirectional_promoter,...,NaN,NaN,YAR035C-A,upstream_gene_variant,2.0,SWH1,YAR042W,upstream_gene_variant,200.0,3.0
GXE_00024,I,192424,GTTTGGATTACCTCT,ATTTGGATTACCTCT,39,2014,0.019000,7|181,4692|3613|168,bidirectional_promoter,...,NaN,NaN,YAR035C-A,upstream_gene_variant,7.0,SWH1,YAR042W,upstream_gene_variant,181.0,3.0
GXE_00025,I,192424,GTTTGGATTACCTCT,G,2,2014,0.000993,8|181,4693|3614|169,bidirectional_promoter,...,NaN,NaN,YAR035C-A,upstream_gene_variant,8.0,SWH1,YAR042W,upstream_gene_variant,181.0,3.0
GXE_00026,I,192427,T,C,902,2000,0.451000,10|192,4695|3616|171,bidirectional_promoter,...,NaN,NaN,YAR035C-A,upstream_gene_variant,10.0,SWH1,YAR042W,upstream_gene_variant,192.0,2.0
GXE_00027,I,192429,G,C,1506,1998,0.754000,12|190,4697|3618|173,bidirectional_promoter,...,NaN,NaN,YAR035C-A,upstream_gene_variant,12.0,SWH1,YAR042W,upstream_gene_variant,190.0,3.0


### Remove singletons & doubletons, but keep those that appear in at least one of the 15 QTL parental strains (Bloom et al., 2019)

In [34]:
# search GVCF and add allele information for parent strains in QTL crosses
chromosome_dict = {'chrI' : 'chromosome1',
                   'chrII': 'chromosome2',
                   'chrIII': 'chromosome3',
                   'chrIV': 'chromosome4',
                   'chrV': 'chromosome5',
                   'chrVI': 'chromosome6',
                   'chrVII': 'chromosome7',
                   'chrVIII': 'chromosome8',
                   'chrIX': 'chromosome9',
                   'chrX': 'chromosome10',
                   'chrXI': 'chromosome11',
                   'chrXII': 'chromosome12',
                   'chrXIII': 'chromosome13',
                   'chrXIV': 'chromosome14',
                   'chrXV': 'chromosome15',
                   'chrXVI': 'chromosome16'}

gxe_variants_file = "gxe_variants.vcf"
i=0
for chromosome, df in gxe_oligos_final.groupby('CHROM'):
    gvcf_file = "/home/users/rang/share/yeast/1011genomes/by_chrom/{}.gvcf.gz".format(chromosome_dict['chr'+chromosome])
    allVars = vcf.Reader(filename=gvcf_file)
    
    for v, row in df.iterrows():
        # find variant in 1011 genomes GVCF
        retrieve = allVars.fetch(chrom=chromosome_dict['chr'+row.CHROM], 
                                 start=row.POS-1,
                                 end=row.POS)
        # select record that matches 
        for r in retrieve:
            if (r.POS==row.POS) & (r.REF==row.REF):
                record = r
                break
                
        # get all alternate alleles at this position
        gxe_oligos_final.loc[v, 'all_alt_alleles'] = ','.join([str(a) for a in record.ALT])
        # get allele calls for strains in yeast_strains_id
        for s in record.samples:
            if (s.sample in yeast_strains_id) and (None not in s.gt_alleles):
                gxe_oligos_final.loc[v, s.sample] = '/'.join(s.gt_alleles)
        
        i+=1
        if i % 500 == 0:
            print(i, "variants processed")
        
# write updated gxe_oligos_final table
gxe_oligos_final.to_csv(gxe_variants_file.replace('.vcf', '_with_parents.txt'), sep='\t')

500 variants processed
1000 variants processed
1500 variants processed
2000 variants processed
2500 variants processed
3000 variants processed
3500 variants processed
4000 variants processed
4500 variants processed
5000 variants processed
5500 variants processed
6000 variants processed


In [35]:
# find singletons and doubletons where at least 1 alt allele found in 15 parental strains used to map QTLs
df = gxe_oligos_final[~( (gxe_oligos_final.loc[:,yeast_strains_id] == '0/0') | (gxe_oligos_final.loc[:,yeast_strains_id].isna()) ).all(axis=1)].query('AC<3')

# check if parental strains carry alt allele that matches variant entry
singletons_doubletons_include_list = []
for v, row in df.iterrows():
    alt_allele_num = row.all_alt_alleles.split(',').index(row.ALT)+1
    if any([str(alt_allele_num) in gt for gt in row[yeast_strains_id].fillna('unknown')]):
        singletons_doubletons_include_list.append(v)
        
# finally, subset gxe_oligos_final to remove singletons and doubletons, unless variant is in singletons_doubletons_include_list
gxe_oligos_final_no_singletons = gxe_oligos_final.query('AC>=3 | var_id.isin(@singletons_doubletons_include_list)')
display(gxe_oligos_final_no_singletons.shape)
display(gxe_oligos_final_no_singletons.head())

(4384, 58)

,CHROM,POS,REF,ALT,AC,AN,AF,upstream_distance_str,downstream_distance_str,region,...,ACG,ACI,ACQ,ACS,ADE,ADF,ADG,ADI,ADR,SACE_MAA
var_id,,,,,,,,,,,,,,,,,,,,,
GXE_00023,I,192419,C,A,6,2020,0.002970,2|200,4687|3608|163,bidirectional_promoter,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
GXE_00024,I,192424,GTTTGGATTACCTCT,ATTTGGATTACCTCT,39,2014,0.019000,7|181,4692|3613|168,bidirectional_promoter,...,0/0,0/0,0/0,0/0,NaN,0/0,0/0,0/0,0/0,0/0
GXE_00026,I,192427,T,C,902,2000,0.451000,10|192,4695|3616|171,bidirectional_promoter,...,1/1,0/0,0/0,0/0,0/0,0/0,0/0,1/1,1/1,0/0
GXE_00027,I,192429,G,C,1506,1998,0.754000,12|190,4697|3618|173,bidirectional_promoter,...,1/1,1/1,1/1,1/1,NaN,1/1,1/1,1/1,1/1,0/0
GXE_00033,I,192466,AAAGGG,A,3,2020,0.001485,50|148,4735|3656|211,bidirectional_promoter,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [36]:
# use QTL/gene coordinates to associate each variant with traits in gxe_qtls
qtls_df = gxe_qtls[['trait', 'chrom', 'start', 'end', 'qtl_left', 'qtl_right']].copy()
qtls_df.loc[:,'chrom'] = qtls_df['chrom'].str.replace('chr', '')
# find gene upstream and downstream boundaries
qtls_df.loc[:, 'gene_left'] = qtls_df['start']-500
qtls_df.loc[:, 'gene_right'] = qtls_df['end']+500
# define search boundaries by either gene or QTL boundary, whichever is larger
qtls_df.loc[:,'left_limit'] = qtls_df[['gene_left', 'qtl_left']].min(axis=1)
qtls_df.loc[:,'right_limit'] = qtls_df[['gene_right', 'qtl_right']].max(axis=1)
qtls_df.loc[:,'left_limit'] = qtls_df['left_limit'].astype(int)
qtls_df.loc[:,'right_limit'] = qtls_df['right_limit'].astype(int)
qtls_df.loc[:, 'region_length'] = qtls_df['right_limit'] - qtls_df['left_limit'] + 1
# remove search boundaries duplicates
qtls_df = qtls_df[['trait', 'chrom', 'left_limit', 'right_limit']].drop_duplicates()

# associate each variant with traits of interest if it falls within QTL/gene search boundaries
gxe_oligos_final_no_singletons.loc[:,'assoc_traits'] = gxe_oligos_final_no_singletons.apply(lambda x: qtls_df.query('chrom==@x.CHROM & left_limit<=@x.POS & right_limit>=@x.POS').trait.unique().tolist(), axis=1)
gxe_oligos_final_no_singletons.loc[:,'assoc_traits_str'] = gxe_oligos_final_no_singletons['assoc_traits'].apply(lambda x: '_'.join(x))

display(gxe_oligos_final_no_singletons.shape)
display(gxe_oligos_final_no_singletons.head())

# write gxe_oligos_final_no_singletons table to file
gxe_oligos_final_no_singletons.to_csv(gxe_variants_file.replace('.vcf', '_final.txt'), sep='\t')

/home/groups/hbfraser/modules/packages/conda/4.6.14/envs/crispey-bar/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/groups/hbfraser/modules/packages/conda/4.6.14/envs/crispey-bar/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


(4384, 60)

,CHROM,POS,REF,ALT,AC,AN,AF,upstream_distance_str,downstream_distance_str,region,...,ACQ,ACS,ADE,ADF,ADG,ADI,ADR,SACE_MAA,assoc_traits,assoc_traits_str
var_id,,,,,,,,,,,,,,,,,,,,,
GXE_00023,I,192419,C,A,6,2020,0.002970,2|200,4687|3608|163,bidirectional_promoter,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,"[Cobalt_Chloride;2mM;2, Fluconazole;100uM;2, N...",Cobalt_Chloride;2mM;2_Fluconazole;100uM;2_Neom...
GXE_00024,I,192424,GTTTGGATTACCTCT,ATTTGGATTACCTCT,39,2014,0.019000,7|181,4692|3613|168,bidirectional_promoter,...,0/0,0/0,NaN,0/0,0/0,0/0,0/0,0/0,"[Cobalt_Chloride;2mM;2, Fluconazole;100uM;2, N...",Cobalt_Chloride;2mM;2_Fluconazole;100uM;2_Neom...
GXE_00026,I,192427,T,C,902,2000,0.451000,10|192,4695|3616|171,bidirectional_promoter,...,0/0,0/0,0/0,0/0,0/0,1/1,1/1,0/0,"[Cobalt_Chloride;2mM;2, Fluconazole;100uM;2, N...",Cobalt_Chloride;2mM;2_Fluconazole;100uM;2_Neom...
GXE_00027,I,192429,G,C,1506,1998,0.754000,12|190,4697|3618|173,bidirectional_promoter,...,1/1,1/1,NaN,1/1,1/1,1/1,1/1,0/0,"[Cobalt_Chloride;2mM;2, Fluconazole;100uM;2, N...",Cobalt_Chloride;2mM;2_Fluconazole;100uM;2_Neom...
GXE_00033,I,192466,AAAGGG,A,3,2020,0.001485,50|148,4735|3656|211,bidirectional_promoter,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,"[Cobalt_Chloride;2mM;2, Fluconazole;100uM;2, N...",Cobalt_Chloride;2mM;2_Fluconazole;100uM;2_Neom...


## Summary of GxE variants

In [37]:
print("Number of variants:", len(gxe_oligos_final_no_singletons))
print("Number of oligos:", int(sum(gxe_oligos_final_no_singletons.num_of_oligos)))
print()
print("Distribution of variants among gxe genes")
print(gxe_oligos_final_no_singletons.Gene_ID.value_counts().filter(gxe_qtls.gene_id.unique()).describe())
print()
print("Distribution of oligos among gxe variants")
print(gxe_oligos_final_no_singletons.num_of_oligos.describe())
print()
print("Proportion of variant annotation classes")
print(gxe_oligos_final_no_singletons.region.value_counts()/len(gxe_oligos_final_no_singletons))

Number of variants: 4384
Number of oligos: 10983

Distribution of variants among gxe genes
count     53.000000
mean      73.245283
std       44.006951
min       10.000000
25%       39.000000
50%       68.000000
75%       98.000000
max      189.000000
Name: Gene_ID, dtype: float64

Distribution of oligos among gxe variants
count    4384.000000
mean        2.505246
std         0.808900
min         2.000000
25%         2.000000
50%         2.000000
75%         3.000000
max         7.000000
Name: num_of_oligos, dtype: float64

Proportion of variant annotation classes
synonymous_variant         0.377965
missense_variant           0.288549
bidirectional_promoter     0.140511
unidirectional_promoter    0.123859
intergenic                 0.055201
frameshift_variant         0.008212
stop_gained                0.003650
inframe_insertion          0.001369
inframe_deletion           0.000456
stop_lost                  0.000228
Name: region, dtype: float64


## Determine number of wells required in matrixed oligo synthesis array
Twist Biosciences synthesizes 121 oligos per well, up to 384 wells per plate.

In [38]:
print('Number of variants per condition(s)')
print(gxe_oligos_final_no_singletons.groupby('assoc_traits_str').size())
print()
print('Number of oligos per condition(s)')
print(gxe_oligos_final_no_singletons.groupby('assoc_traits_str').num_of_oligos.sum())
print()

# count variants and oligos per trait
for trait in gxe_qtls.trait.unique():
    print("Trait:", trait)
    print('Number of variants:', len(gxe_oligos_final_no_singletons[gxe_oligos_final_no_singletons.assoc_traits.apply(lambda x: trait in x)]) )
    print('Number of oligos:', int(gxe_oligos_final_no_singletons[gxe_oligos_final_no_singletons.assoc_traits.apply(lambda x: trait in x)]['num_of_oligos'].sum()) )
    print()

Number of variants per condition(s)
assoc_traits_str
Caffeine;15mM;2                                                879
Caffeine;15mM;2_Cobalt_Chloride;2mM;2                           41
Caffeine;15mM;2_Fluconazole;100uM;2_Neomycin;5mg/mL;2          163
Caffeine;15mM;2_Fructose;;1                                      1
Caffeine;15mM;2_Neomycin;5mg/mL;2                                5
Cobalt_Chloride;2mM;2                                          611
Cobalt_Chloride;2mM;2_Fluconazole;100uM;2                       74
Cobalt_Chloride;2mM;2_Fluconazole;100uM;2_Fructose;;1            4
Cobalt_Chloride;2mM;2_Fluconazole;100uM;2_Neomycin;5mg/mL;2    172
Cobalt_Chloride;2mM;2_Fructose;;1                               70
Cobalt_Chloride;2mM;2_Fructose;;1_Neomycin;5mg/mL;2             48
Fluconazole;100uM;2                                            531
Fluconazole;100uM;2_Fructose;;1                                 32
Fructose;;1                                                    632
Lithium_C

In [39]:
# count OLIGOS in each variant prefix set
display(gxe_oligos_final_no_singletons.groupby(lambda x: x[:3]).num_of_oligos.sum().sort_values(ascending=False))

# determine number of pools to use
num_of_oligos = int(gxe_oligos_final_no_singletons.num_of_oligos.sum())
min_pool_size = 109 # allows 12 technical oligos
max_pool_size = 118 # allows 3 technical oligos
complete_pool_size = 121 # maximum oligos synthesized per pool

# find number of pools to use
find_pool_size_options(num_of_oligos, min_pool_size, max_pool_size, complete_pool_size)

GXE    10983.0
Name: num_of_oligos, dtype: float64

Total number of oligos to fit: 10983


(93, 100)

Use 93 pools:
Number of oligos per pool: 118
Number of oligos leftover: 9
Number of technical oligos: 279

Use 94 pools:
Number of oligos per pool: 116
Number of oligos leftover: 79
Number of technical oligos: 470

Use 95 pools:
Number of oligos per pool: 115
Number of oligos leftover: 58
Number of technical oligos: 570

Use 96 pools:
Number of oligos per pool: 114
Number of oligos leftover: 39
Number of technical oligos: 672

Use 97 pools:
Number of oligos per pool: 113
Number of oligos leftover: 22
Number of technical oligos: 776

Use 98 pools:
Number of oligos per pool: 112
Number of oligos leftover: 7
Number of technical oligos: 882

Use 99 pools:
Number of oligos per pool: 110
Number of oligos leftover: 93
Number of technical oligos: 1089

Use 100 pools:
Number of oligos per pool: 109
Number of oligos leftover: 83
Number of technical oligos: 1200



array([ 93,  94,  95,  96,  97,  98,  99, 100])

# Selecting variants in ergosterol pathway genes for GxE project
Screen natural variation in ergosterol genes and test for GxE effects

In [40]:
# set working directory
working_dir = "/home/users/rang/scratch/yeast/ergosterol/"
# list of ergosterol genes
ergosterol_genes_file = working_dir+"ergosterol_biosynthetic_process_genes.txt"
# summary of genetic interactions output file
genetic_interactions_summary_file = "/home/users/rang/scratch/yeast/genetic_interactions/costanzo_2016/genetic_interactions_summary_by_gene.txt"
os.chdir(working_dir)

In [41]:
# read in ergosterol genes table
ergosterol_genes = pd.read_csv(ergosterol_genes_file, sep='\t')
ergosterol_genes = ergosterol_genes.rename(columns={'Gene Systematic Name':'gene_id', 'Gene':'gene_name'}).set_index('gene_id')

# annotate ergosterol genes
ergosterol_genes = ergosterol_genes.merge(gene_info_df.drop('gene_name', axis=1), how='left', left_index=True, right_index=True)

# add genetic interaction data
genetic_interactions = pd.read_csv(genetic_interactions_summary_file, sep='\t', index_col=0)
ergosterol_genes = ergosterol_genes.merge(genetic_interactions, how='left', left_index=True, right_index=True)
display(ergosterol_genes.shape)
display(ergosterol_genes.head())

(26, 12)

,gene_name,Details,Mutant Information,Breslow fitness,pathway,chrom,start,end,strand,description,positive_interactions,negative_interactions
gene_id,,,,,,,,,,,,
YPL028W,ERG10,Media: minimal medium Details: Relative fitnes...,reduction of function,0.935,upstream,chrXVI,498096,499292,+,Acetyl-CoA C-acetyltransferase (acetoacetyl-Co...,152.0,263.0
YMR208W,ERG12,Media: minimal medium Details: Relative fitnes...,reduction of function,1.016,upstream,chrXIII,684467,685798,+,"Mevalonate kinase, acts in the biosynthesis of...",27.0,107.0
YML126C,ERG13,Media: minimal medium Details: Relative fitnes...,reduction of function,0.929,upstream,chrXIII,19060,20535,-,3-hydroxy-3-methylglutaryl-CoA (HMG-CoA) synth...,206.0,316.0
YJL167W,ERG20,Media: minimal medium Details: Relative fitnes...,reduction of function,0.995,upstream,chrX,105014,106072,+,"Farnesyl pyrophosphate synthetase, has both di...",NaN,NaN
YMR220W,ERG8,Media: minimal medium Details: Relative fitnes...,reduction of function,0.953,upstream,chrXIII,712316,713671,+,"Phosphomevalonate kinase, an essential cytosol...",102.0,150.0


## Scan 1,011 genomes VCF for variants in and around gene of interest

In [42]:
# search ergosterol genes for variants in each gvcf
chromosome_dict = {'chrI' : 'chromosome1',
                   'chrII': 'chromosome2',
                   'chrIII': 'chromosome3',
                   'chrIV': 'chromosome4',
                   'chrV': 'chromosome5',
                   'chrVI': 'chromosome6',
                   'chrVII': 'chromosome7',
                   'chrVIII': 'chromosome8',
                   'chrIX': 'chromosome9',
                   'chrX': 'chromosome10',
                   'chrXI': 'chromosome11',
                   'chrXII': 'chromosome12',
                   'chrXIII': 'chromosome13',
                   'chrXIV': 'chromosome14',
                   'chrXV': 'chromosome15',
                   'chrXVI': 'chromosome16',}

genes_to_process = []
for chrom in chromosome_dict.keys():
    genes_df = ergosterol_genes.query('chrom==@chrom').sort_values('start')
    gvcf_file = "/home/users/rang/share/yeast/1011genomes/by_chrom/{}.gvcf.gz".format(chromosome_dict[chrom])
    genes_to_process.append((genes_df, gvcf_file, chromosome_dict))
    
# consolidate candidate variants
num_of_cores = min(len(os.sched_getaffinity(0)), len(genes_to_process))
with mp.Pool(num_of_cores) as pool:
    results = pool.starmap(select_variants, genes_to_process)
results = pd.concat(results).reset_index(drop=True)

# write to new vcf file
ergosterol_variants_file = "ergosterol_variants.vcf"
template = vcf.Reader(filename=genes_to_process[0][1])
# change chromosome naming convention to a single Roman numeral (follows ref genome, VEP formatting)
chromosome_dict_rev = {v:k for k, v in chromosome_dict.items()} 
template.contigs = OrderedDict((chromosome_dict_rev[k][3:], v._replace(id=chromosome_dict_rev[k][3:])) for k, v in template.contigs.items())
# remove 1011 genomes samples data
template.samples=[]

output = vcf.Writer(open(ergosterol_variants_file, 'w'), template)
for record in results:
    # update chromosome naming
    record.CHROM = chromosome_dict_rev[record.CHROM][3:]
    # write to file
    output.write_record(record)
output.close()

## Filter variants for those that can be targeted for REF to ALT allele editing
Use a 60 bp window to ensure the same sequence (ref allele +- 30bp) can be found in four yeast strains

In [43]:
# path to genome fasta files
ref_genome_fasta = "/home/users/rang/yeast/genomes/Saccharomyces_cerevisiae.R64-1-1.dna.chromosome.I.fa"
rm_genome_fasta = "/home/users/rang/yeast/genomes/RM11-1A_SGD_2015_JRIP00000000.fsa"
yjm_genome_fasta = "/home/users/rang/yeast/genomes/YJM789_Stanford_2007_AAFW02000000_highQuality31.fsa"
yps_genome_fasta = "/home/users/rang/yeast/genomes/YPS128.genome.fa"

# load genome files
ref_genome = SeqIO.to_dict(SeqIO.parse(ref_genome_fasta, "fasta"))
rm_genome = SeqIO.to_dict(SeqIO.parse(rm_genome_fasta, "fasta"))
yjm_genome = SeqIO.to_dict(SeqIO.parse(yjm_genome_fasta, "fasta"))
yps_genome = SeqIO.to_dict(SeqIO.parse(yps_genome_fasta, "fasta"))
genomes_list = [ref_genome, rm_genome, yjm_genome, yps_genome] # IMPORTANT: SET REFERENCE GENOME AS FIRST IN LIST

In [44]:
# path to input and output VCF files
ergosterol_variants_file = "ergosterol_variants.vcf"
ergosterol_variants_select_file = ergosterol_variants_file.replace('.vcf', '_select.vcf')

threshold = 10 # threshold value for searching NGGs
window = 61 # size of flanking sequences to search for unique hits

# read in all candidate variants to filter
ergosterol_variants = vcf.Reader(filename=ergosterol_variants_file)
ergosterol_variants = [rec for rec in ergosterol_variants]
# split variants into chunks for multiprocessing
n = len(ergosterol_variants)//len(os.sched_getaffinity(0)) + 1
ergosterol_variants = [ergosterol_variants[i:i+n] for i in range(0, len(ergosterol_variants), n)]

# find targetable variants (TGT info field indicates which genomes variant is targetable in)
to_process = [(rec_list, genomes_list, threshold, window) for rec_list in ergosterol_variants]
with mp.Pool(len(os.sched_getaffinity(0))) as pool:
    result = pool.starmap(find_targetable_variants, to_process) 

# write targetable variants to file
header = vcf.Reader(filename=ergosterol_variants_file)
# add "TGT" info field into VCF header
header_info = namedtuple('Info', ['id', 'num', 'type', 'desc', 'source', 'version'])
header_info_tgt = header_info('TGT', '.', 'Integer', 'Targetability of variant in tested genomes', None, None)
header.infos['TGT'] = header_info_tgt
# write to file
ergosterol_variants_select = vcf.Writer(open(ergosterol_variants_select_file, 'w'), header)
to_write = [rec for rec_list in result for rec in rec_list]
for rec in to_write:
    ergosterol_variants_select.write_record(rec)
ergosterol_variants_select.close()

## Filter variants by allele frequency
1. For variants with multiple alternate alleles, split variant entry into separate lines in VCF for library design - allows oligo design pipeline to synthesize separate oligos for each type of REF-to-ALT edit

Variants are assigned an ID prefix "ERG" or "EG_" to denote their use in the CRISPEY-BAR GxE project

In [45]:
# select variants by allele frequency (currently disabled singleton/doubleton filter)
### if multiple alternate alleles, SPLIT VARIANT ENTRY INTO SEPARATE ENTRIES in VCF (useful for oligo design pipeline)
### assign unitl variant ID prefix and number (use format VAR_XXXXX)

# candidate variants editable in all strains
ergosterol_variants_select = vcf.Reader(filename=ergosterol_variants_select_file)

# open output file
ergosterol_variants_select_AFfilter_file = ergosterol_variants_select_file.replace('.vcf', '_AFfilter.vcf')
ergosterol_variants_select_AFfilter = vcf.Writer(open(ergosterol_variants_select_AFfilter_file, 'w'), ergosterol_variants_select)

var_counter = {}
for record in ergosterol_variants_select:        
    # (optional step to skip singletons and doubletons -- DISABLED by setting ac<1)
    if all([ac<1 for ac in record.INFO['AC']]):
        continue
    # if at least one alternate allele meets minimum count, write to VCF
    else:
        alt = record.ALT[:]
        ac = record.INFO['AC'][:]
        af = record.INFO['AF'][:]
        mleac = record.INFO['MLEAC'][:]
        mleaf = record.INFO['MLEAF'][:]
        
        # split each alt allele into its own variant entry (useful if a locus has multiple possible alleles)
        for i in range(len(alt)):
            record.ALT = [alt[i]]
            record.INFO['AC'] = [ac[i]]
            record.INFO['AF'] = [af[i]]
            record.INFO['MLEAC'] = [mleac[i]]
            record.INFO['MLEAF'] = [mleaf[i]]

            # inspect TGT field and assign variant ID prefix
            ### if tgt field contains all 1 (i.e. targetable by all queried genomes), assign "ERG"
            ### otherwise, assign unique prefix "EG-" with the last digit determined by converting binary to hexadecimal (works well for up to 4 genomes)
            if all([i==1 for i in record.INFO['TGT']]):
                var_id_prefix = "ERG"
            else:
                binary_str = ''.join([str(x) for x in record.INFO['TGT']])
                hex_digit = hex(int(binary_str, 2))[2:].upper()
                var_id_prefix = "EG{}".format(hex_digit)

            # add to counter
            if var_id_prefix not in var_counter:
                var_counter[var_id_prefix] = 1
            else:
                var_counter[var_id_prefix] += 1

            # add unique variant ID (follow format VAR_XXXXX)
            record.ID='{}_{:05d}'.format(var_id_prefix, var_counter[var_id_prefix])

            # write to output
            ergosterol_variants_select_AFfilter.write_record(record)
            
ergosterol_variants_select_AFfilter.close()

## Submit ergosterol_variants_select_AFfilter VCF to VEP to get annotations, and download VEP output in TXT format (vep_output_filename).
http://uswest.ensembl.org/Saccharomyces_cerevisiae/Tools/VEP

In [46]:
# add annotations to variants and remove variants that fall in non-ergosterol genes.
ergosterol_variants_file = "ergosterol_variants.vcf"
ergosterol_variants_select_file = ergosterol_variants_file.replace('.vcf', '_select.vcf')

# get VEP annotations and remove variants that fall in non-ergosterol pathway genes
ergosterol_variants_select_AFfilter_file = ergosterol_variants_select_file.replace('.vcf', '_AFfilter.vcf')
vep_output_filename = ergosterol_variants_select_AFfilter_file.replace('.vcf', '_VEPoutput.txt')
variants_annotated_filename = vep_output_filename.replace('_VEPoutput.txt', '_annotated.txt')
variants_annotated_df = annotate_variants_by_VEPoutput(ergosterol_variants_select_AFfilter_file, 
                                                       vep_output_filename, 
                                                       gff_file, 
                                                       variants_annotated_filename, 
                                                       id_colname='var_id')

In [47]:
# read in annotated file
variants_annotated_df = pd.read_csv(variants_annotated_filename, sep='\t', index_col=0)

# remove variants that fall within genes, but are not in the approved genes list
intergenic_regions_list = ['bidirectional_promoter', 'unidirectional_promoter', 'intergenic']
ergosterol_design_oligos = variants_annotated_df.query('(Gene_ID.isin(@ergosterol_genes.index) | region.isin(@intergenic_regions_list))').copy()
display(ergosterol_design_oligos.shape)
display(ergosterol_design_oligos.head())

(4082, 41)

,CHROM,POS,REF,ALT,AC,AN,AF,upstream_distance_str,downstream_distance_str,region,...,Codons,DOMAINS,closest_gene1_Gene_Name,closest_gene1_Gene_ID,closest_gene1_Annotation,closest_gene1_Distance,closest_gene2_Gene_Name,closest_gene2_Gene_ID,closest_gene2_Annotation,closest_gene2_Distance
var_id,,,,,,,,,,,,,,,,,,,,,
ERG_00002,V,237121,G,A,9,2022,0.004451,2,2381|1630|1339|449|2911,unidirectional_promoter,...,NaN,NaN,SAH1,YER043C,upstream_gene_variant,2.0,ERG28,YER044C,downstream_gene_variant,449.0
ERG_00003,V,237122,G,A,9,2022,0.004451,3,2382|1631|1338|448|2910,unidirectional_promoter,...,NaN,NaN,SAH1,YER043C,upstream_gene_variant,3.0,ERG28,YER044C,downstream_gene_variant,448.0
ERG_00004,V,237136,T,C,26,2022,0.013000,17,2396|1645|1324|434|2896,unidirectional_promoter,...,NaN,NaN,SAH1,YER043C,upstream_gene_variant,17.0,ERG28,YER044C,downstream_gene_variant,434.0
EGE_00001,V,237149,A,C,43,2022,0.021000,30,2409|1658|1311|421|2883,unidirectional_promoter,...,NaN,NaN,SAH1,YER043C,upstream_gene_variant,30.0,ERG28,YER044C,downstream_gene_variant,421.0
EGE_00002,V,237151,T,C,2,2022,0.000989,32,2411|1660|1309|419|2881,unidirectional_promoter,...,NaN,NaN,SAH1,YER043C,upstream_gene_variant,32.0,ERG28,YER044C,downstream_gene_variant,419.0


In [48]:
# write variants to ergosterol_design_oligos VCF as a preliminary list to design oligos for
ergosterol_design_oligos_vcf_file = ergosterol_variants_file.replace('.vcf', '_design_oligos_initial.vcf')

ergosterol_variants_select_AFfilter = vcf.Reader(filename=ergosterol_variants_select_AFfilter_file)
ergosterol_design_oligos_vcf = vcf.Writer(open(ergosterol_design_oligos_vcf_file, 'w'), ergosterol_variants_select_AFfilter)
for record in ergosterol_variants_select_AFfilter:
    if record.ID in ergosterol_design_oligos.index:
        ergosterol_design_oligos_vcf.write_record(record)
ergosterol_design_oligos_vcf.close()

## Run oligo design script (crispey3_design_library.ipynb) with these variants to identify which can be targeted by at least two unique guides

In [49]:
oligo_file = os.path.expanduser("~/crispey3/initial_design/Output/all_SNPs_ergosterol_initial_GG_9bp_OLIGO.tab")
ergosterol_oligo_df = pd.read_csv(oligo_file, sep='\t')

ergosterol_design_oligos = pd.read_csv("/home/users/rang/scratch/yeast/ergosterol/ergosterol_variants_select_AFfilter_annotated.txt", sep='\t', index_col=0)
# remove variants that fall within genes, but are not in the approved genes list
intergenic_regions_list = ['bidirectional_promoter', 'unidirectional_promoter', 'intergenic']
ergosterol_design_oligos = ergosterol_design_oligos.query('(Gene_ID.isin(@ergosterol_genes.index) | region.isin(@intergenic_regions_list))').copy()

ergosterol_design_oligos['num_of_oligos'] = ergosterol_oligo_df.groupby('var_id').apply(len)
ergosterol_oligos_final = ergosterol_design_oligos.query('num_of_oligos>1').copy()
display(ergosterol_oligos_final.shape)
display(ergosterol_oligos_final.head())

# write ergosterol_oligos_final table to file
ergosterol_oligos_final.to_csv("/home/users/rang/scratch/yeast/ergosterol/ergosterol_variants_final.txt", sep='\t')

(1580, 42)

,CHROM,POS,REF,ALT,AC,AN,AF,upstream_distance_str,downstream_distance_str,region,...,DOMAINS,closest_gene1_Gene_Name,closest_gene1_Gene_ID,closest_gene1_Annotation,closest_gene1_Distance,closest_gene2_Gene_Name,closest_gene2_Gene_ID,closest_gene2_Annotation,closest_gene2_Distance,num_of_oligos
var_id,,,,,,,,,,,,,,,,,,,,,
ERG_00003,V,237122,G,A,9,2022,0.004451,3,2382|1631|1338|448|2910,unidirectional_promoter,...,NaN,SAH1,YER043C,upstream_gene_variant,3.0,ERG28,YER044C,downstream_gene_variant,448.0,2.0
ERG_00004,V,237136,T,C,26,2022,0.013000,17,2396|1645|1324|434|2896,unidirectional_promoter,...,NaN,SAH1,YER043C,upstream_gene_variant,17.0,ERG28,YER044C,downstream_gene_variant,434.0,2.0
EGE_00008,V,237191,T,TG,661,2022,0.327000,72,2451|1700|1268|378|2840,unidirectional_promoter,...,NaN,SAH1,YER043C,upstream_gene_variant,72.0,ERG28,YER044C,downstream_gene_variant,378.0,2.0
EGE_00009,V,237191,T,TA,6,2022,0.002967,72,2451|1700|1268|378|2840,unidirectional_promoter,...,NaN,SAH1,YER043C,upstream_gene_variant,72.0,ERG28,YER044C,downstream_gene_variant,378.0,2.0
EGE_00010,V,237196,A,T,69,2022,0.034000,77,2456|1705|1264|374|2836,unidirectional_promoter,...,NaN,SAH1,YER043C,upstream_gene_variant,77.0,ERG28,YER044C,downstream_gene_variant,374.0,2.0


## Summary of ergosterol variants

In [50]:
print("Number of variants:", len(ergosterol_oligos_final))
print("Number of oligos:", int(sum(ergosterol_oligos_final.num_of_oligos)))
print()
print("Distribution of variants among ergosterol genes")
print(ergosterol_oligos_final.Gene_ID.value_counts().filter(ergosterol_genes.index).describe())
print()
print("Distribution of oligos among ergosterol variants")
print(ergosterol_oligos_final.num_of_oligos.describe())
print()
print("Proportion of variant annotation classes")
print(ergosterol_oligos_final.region.value_counts()/len(ergosterol_oligos_final))

Number of variants: 1580
Number of oligos: 3968

Distribution of variants among ergosterol genes
count     26.000000
mean      49.384615
std       26.023185
min       19.000000
25%       32.250000
50%       42.000000
75%       57.000000
max      114.000000
Name: Gene_ID, dtype: float64

Distribution of oligos among ergosterol variants
count    1580.000000
mean        2.511392
std         0.858882
min         2.000000
25%         2.000000
50%         2.000000
75%         3.000000
max         7.000000
Name: num_of_oligos, dtype: float64

Proportion of variant annotation classes
synonymous_variant         0.372152
missense_variant           0.236076
unidirectional_promoter    0.183544
bidirectional_promoter     0.149367
intergenic                 0.053165
frameshift_variant         0.002532
stop_gained                0.001899
inframe_deletion           0.001266
Name: region, dtype: float64


## Determine number of wells required in matrixed oligo synthesis array
Twist Biosciences synthesizes 121 oligos per well, up to 384 wells per plate.

In [51]:
# count OLIGOS in each variant prefix set
display(ergosterol_oligos_final.groupby(lambda x: x[:3]).num_of_oligos.sum().sort_values(ascending=False))

# determine number of pools to use
num_of_oligos = int(ergosterol_oligos_final.num_of_oligos.sum())
min_pool_size = 109 # allows 12 technical oligos
max_pool_size = 118 # allows 3 technical oligos
complete_pool_size = 121 # maximum oligos synthesized per pool

# find number of pools to use
find_pool_size_options(num_of_oligos, min_pool_size, max_pool_size, complete_pool_size)

ERG    2479.0
EGE     469.0
EG8     253.0
EGB     209.0
EG9     196.0
EGC     193.0
EGD     134.0
EGA      35.0
Name: num_of_oligos, dtype: float64

Total number of oligos to fit: 3968


(33, 36)

Use 33 pools:
Number of oligos per pool: 118
Number of oligos leftover: 74
Number of technical oligos: 99

Use 34 pools:
Number of oligos per pool: 116
Number of oligos leftover: 24
Number of technical oligos: 170

Use 35 pools:
Number of oligos per pool: 113
Number of oligos leftover: 13
Number of technical oligos: 280

Use 36 pools:
Number of oligos per pool: 110
Number of oligos leftover: 8
Number of technical oligos: 396



array([33, 34, 35, 36])